In [ ]:
# !pip install shap torch torchvision


In [1]:
import sys
from SVERL_icml_2023.portfolio_DRL.data_function import *
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader
from SVERL_icml_2023.portfolio_DRL.create_model import *
from sklearn.model_selection import TimeSeriesSplit
from SVERL_icml_2023.shapley import Shapley
import shap
sys.path.append("E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023")
# from portfolio_DRL.create_model import StockPredictWrapper

In [ ]:

# Example usage
input_dir = "E:\XAI\RL_with_SHAP\pythonProject1\SVERL_icml_2023\portfolio_DRL"  # Use the specified directory
ticker_file = os.path.join(input_dir, "tickers.txt")  # Text file containing tickers, one per line
factor_file = os.path.join(input_dir, "FF_factors.csv")  # CSV file containing factor returns
start_date = "1999-01-01"
end_date = "2025-03-31"
output_csv = os.path.join(input_dir, "stock_prices.csv")

try:
    tickers = load_tickers_from_file(ticker_file)
    price_data = download_stock_prices(tickers, start_date, end_date, output_csv)
    daily_returns = calculate_daily_returns(price_data, return_type='arithmetic')
    daily_returns.to_csv(os.path.join(input_dir, "daily_returns.csv"))
    factor_returns = read_factor_returns(factor_file, daily_returns)
    factor_returns.to_csv(os.path.join(input_dir, "aligned_factors.csv"))
    print("Factor returns alignment completed.")
except Exception as e:
    print(f"An error occurred: {e}")

factors = pd.read_csv(f"{input_dir}/aligned_factors.csv", index_col=0, parse_dates=True)
returns_raw = pd.read_csv(f"{input_dir}/daily_returns.csv", index_col=0, parse_dates=True)


# Remove NaN values and align dates
factors = factors.dropna()
returns = returns_raw.dropna()

# Align dates between factors and returns
dates = factors.index.intersection(returns.index)
factors_raw = factors.loc[dates]
returns_raw = returns.loc[dates]
factors_raw = factors_raw.iloc[:, :-1]
factors_raw

In [2]:
input_dir = "E:\XAI\RL_with_SHAP\pythonProject1\SVERL_icml_2023\portfolio_DRL"  # Use the specified directory
ticker_file = os.path.join(input_dir, "tickers.txt")  # Text file containing tickers, one per line
factor_file = os.path.join(input_dir, "FF_factors.csv")  # CSV file containing factor returns
start_date = "1999-01-01"
end_date = "2024-11-29"
output_csv = os.path.join(input_dir, "stock_prices.csv")

try:
    tickers = load_tickers_from_file(ticker_file)
    price_data = download_stock_prices(tickers, start_date, end_date, output_csv)
    daily_returns = calculate_daily_returns(price_data, return_type='arithmetic')
    daily_returns.to_csv(os.path.join(input_dir, "daily_returns_DIA_ETF.csv"))
    factor_returns = read_factor_returns(factor_file, daily_returns)
    factor_returns.to_csv(os.path.join(input_dir, "aligned_factors.csv"))
    print("Factor returns alignment completed.")
except Exception as e:
    print(f"An error occurred: {e}")



Removed tickers with insufficient data: CRM, GS
Data saved to E:\XAI\RL_with_SHAP\pythonProject1\SVERL_icml_2023\portfolio_DRL\stock_prices.csv
Factor returns alignment completed.


In [5]:
factor_file = os.path.join(input_dir, "FF_factors.csv")  # CSV file containing factor returns
daily_returns = pd.read_csv(f"{input_dir}/daily_returns_DIA_ETF.csv", index_col=0, parse_dates=True)
factor_returns = read_factor_returns(factor_file, daily_returns)
factor_returns.to_csv(os.path.join(input_dir, "aligned_factors.csv"))

In [ ]:
import time

def download_stock_prices(tickers, start_date, end_date, output_file):
    """
    Download historical stock prices for the given tickers with rate-limit handling.

    Parameters:
    tickers (list): List of stock tickers.
    start_date (str): Start date in 'YYYY-MM-DD' format.
    end_date (str): End date in 'YYYY-MM-DD' format.
    output_file (str): Path to save the resulting CSV file.

    Returns:
    pd.DataFrame: DataFrame containing the daily price data with tickers as columns and dates as index.
    """
    if not tickers:
        raise ValueError("Ticker list is empty.")

    all_data = {}

    for ticker in tickers:
        print(f"Downloading data for {ticker}...")

        success = False
        retries = 5  # Retry up to 5 times if rate-limited

        while not success and retries > 0:
            try:
                data = yf.download(ticker, start=start_date, end=end_date)

                if 'Close' in data.columns:
                    data = data[['Close']]
                    data.columns = [ticker]  # Rename column to ticker
                    all_data[ticker] = data
                    success = True
                else:
                    print(f"No 'Close' data available for {ticker}. Skipping.")

            except yf.YFRateLimitError as e:
                print(f"Rate limit hit for {ticker}, retrying in 10 seconds...")
                time.sleep(10)  # Wait for 10 seconds before retrying
                retries -= 1

            except Exception as e:
                print(f"Failed to download data for {ticker}: {e}")
                break  # Break if error is not rate limit

        time.sleep(2)  # Add 2-second delay to prevent frequent requests

    if not all_data:
        raise ValueError("No stock data was downloaded.")

    combined_data = pd.concat(all_data.values(), axis=1)
    combined_data.to_csv(output_file)
    print(f"Data saved to {output_file}")

    return combined_data
tickers = load_tickers_from_file(ticker_file)
combined_data = download_stock_prices(tickers, start_date, end_date, output_csv)

In [ ]:
# returns = returns_raw[["MMM"]]

assert not factors_raw.isnull().values.any(), "Factors data contains NaN values"
assert not returns_raw.isnull().values.any(), "Returns data contains NaN values"
assert np.isfinite(factors_raw.values).all(), "Factors data contains infinity"
assert np.isfinite(returns_raw.values).all(), "Returns data contains infinity"

In [ ]:
Nday_diff = 60  # Define the future prediction horizon for shift days
returns_raw = returns_raw.iloc[:, :1]

# Remove NaN values and align dates
factors_raw = factors_raw.dropna()
returns_raw = returns_raw.dropna()
# factors_raw = pd.concat([factors_raw, returns_raw], axis=1)
# Align dates between factors_raw and returns_raw
dates = factors_raw.index.intersection(returns_raw.index)
factors_raw = factors_raw.loc[dates]
returns_raw = returns_raw.loc[dates]

# Shift returns to generate target variable (N-day future return)
returns_raw_Y = returns_raw.shift(-Nday_diff).dropna()

# Align all datasets with available dates in returns_raw_Y
dates_final = factors_raw.index.intersection(returns_raw_Y.index)
factors_raw = factors_raw.loc[dates_final]
returns_raw = returns_raw.loc[dates_final]
returns_raw_Y = returns_raw_Y.loc[dates_final]

N = 60 
sequence_length = 20
batch_size = 100
num_epochs = 100
learning_rate = 0.001
rolling_window_train = 240  # Number of training samples
rolling_window_test = 60   # Number of testing samples
prediction_window = 60     # Number of prediction samples


# sequence_length = 20
# batch_size = 32
# num_epochs = 25
# learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Record results
results = []
prediction_records = []
shapley_records = []
shapley_metrics_df = []
all_test_losses = []
test_results =[]
prediction_results = []
combined_test_result = []

for stock in returns_raw.columns:
    print(f"Training for stock: {stock}")
    stock_returns = returns_raw_Y[[stock]]  # Select the stock column
    factors_raw_X =  pd.concat([factors_raw, stock_returns], axis=1)
    # Rolling split: Train (M days), Test (N days), Predict (W days)
    total_samples = len(factors_raw_X)
        # Initialize the rolling split index
    start = 0

    while start + rolling_window_train + prediction_window + prediction_window <= total_samples:
        train_start = start
        train_end = start + rolling_window_train

        test_start = train_end
        test_end = test_start + rolling_window_test

        predict_start = test_end  # Predictions start right after training
        predict_end = predict_start + prediction_window

        # Extract the actual date indices
        train_dates = (factors_raw_X.iloc[train_start].name, factors_raw_X.iloc[train_end - 1].name)
        test_dates = (factors_raw_X.iloc[test_start].name, factors_raw_X.iloc[test_end - 1].name)
        predict_dates = (factors_raw_X.iloc[predict_start].name, factors_raw_X.iloc[predict_end - 1].name)

        # Print the selected date ranges
        print(f"Training dates: {train_dates}")
        print(f"Testing dates: {test_dates}")
        print(f"Prediction dates: {predict_dates}")

        # # Split data into training, testing, and prediction sets
        train_factors = factors_raw_X.iloc[train_start:train_end]
        train_returns = stock_returns.iloc[train_start:train_end]
        test_factors = factors_raw_X.iloc[test_start:test_end]
        test_returns = stock_returns.iloc[test_start:test_end]
        predict_factors = factors_raw_X.iloc[predict_start:predict_end]
        predict_returns = stock_returns.iloc[predict_start:predict_end]
        # 
        # # Normalize train and test data
        train_factors_mean, train_factors_std = train_factors.mean(), train_factors.std()
        train_returns_mean, train_returns_std = train_returns.mean(), train_returns.std()

        # Avoid division by zero
        train_factors_std.replace(0, 1e-8, inplace=True)
        train_returns_std.replace(0, 1e-8, inplace=True)
        
        train_factors_norm = (train_factors - train_factors_mean) / train_factors_std
        train_returns_norm = (train_returns - train_returns_mean) / train_returns_std
        test_factors_norm = (test_factors - train_factors_mean) / train_factors_std
        test_returns_norm = (test_returns - train_returns_mean) / train_returns_std
        predict_factors_norm = (predict_factors - train_factors_mean) / train_factors_std
        predict_returns_norm = (predict_returns - train_returns_mean) / train_returns_std
        print(f'test_factors_norm dim: {test_factors_norm.shape}')
        
        # # Initialize dataset and dataloader
        dataset = StockDataset(train_factors_norm, train_returns_norm, sequence_length, N=N)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        # 
        # # Model initialization
        d_model = train_factors.shape[1]
        print(f'd_model: {d_model}')

        num_heads = min(4, d_model)  # Ensure num_heads does not exceed d_model and d_model is divisible by num_heads
        while d_model % num_heads != 0:
            num_heads -= 1
        # print(f"d_model: {d_model} num_heads: {num_heads} sequence_length: {sequence_length} and train factor return dim: {train_factors.shape[1]} {train_returns.shape[1]}")
        model = StockPredictAgent(d_model=d_model, num_heads=num_heads, sequence_length=sequence_length, N=N).to(device)
        # 
        # # Optimizer and loss function
        optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
        criterion = nn.SmoothL1Loss(beta=1.0)  # Huber Loss for stability        
        # 
        # # Training the model
        # print(f"Training window: {train_start} to {train_end}")
        # print(f"Expected FC Layer Input Shape: {sequence_length * d_model}")
        # print(f"Flattened Tensor Shape: {batch_size, sequence_length * d_model}")
        train_model_with_logging(model, dataloader, optimizer, criterion,scheduler, num_epochs, device)
        
        # Convert to PyTorch tensors
        test_factors_tensor = torch.tensor(test_factors_norm.values, dtype=torch.float32).to(device).unsqueeze(0)
        predict_factors_tensor = torch.tensor(predict_factors_norm.values, dtype=torch.float32).to(device).unsqueeze(0)
        
        # print(f'test_factors_tensor: {test_factors_tensor.shape}')
        # Run inference on test data
        model.eval()
        with torch.no_grad():
            test_predictions_norm = model(test_factors_tensor).cpu().numpy().flatten()
            future_predictions_norm = model(predict_factors_tensor).cpu().numpy().flatten()
            
            # test_output = model(torch.tensor(test_factors_np, dtype=torch.float32).to(device))
            # print(f"Model output shape: {test_output.shape}")  # Expected: (1, 20) for 20 future returns
        # print(f'test_factors_tensor: {test_factors_tensor.shape}')
        # Revert normalization for test and predicted returns
        test_predictions = test_predictions_norm * train_returns_std.item() + train_returns_mean.item()
        
        
        # 
        # # start SHAP
        # Prepare your input: (1 sample, 20 days, 6 features)
        test_factors_np = test_factors_norm.values.astype('float32')
        test_factors_np = test_factors_np.reshape(1, rolling_window_test, d_model)  # Reshaping for 1 sample with 20 days and 7 features
        # print(f'test_factors_np dim: {test_factors_np.shape}')
        # 
        # # Use a subset of the training data as the background dataset
        background_data = torch.tensor(train_factors_norm.values.astype('float32')).to(device)
        background_data = background_data.reshape(-1, rolling_window_test, d_model)  # Ensure same shape as model input
        # print(f'background_data dim: {background_data.shape}')
        # Ensure background_data is a tensor and avoid redundant torch.tensor() calls
        if not isinstance(background_data, torch.Tensor):
            background_data = torch.tensor(background_data, dtype=torch.float32).to(device)
        else:
            background_data = background_data.to(device)  # Just move to device without re-wrapping
        
        # Initialize SHAP GradientExplainer
        explainer = shap.GradientExplainer(model, background_data)

        # Compute SHAP values for the test input
        shap_values = explainer.shap_values(torch.tensor(test_factors_np, dtype=torch.float32).to(device), ranked_outputs=1)
        # Aggregate SHAP values across outputs and time
        # Verify SHAP values shape
        # print(f"SHAP values shape: {shap_values[0].shape}")  # Expected: (1, 20, 6)
        # print(f"test factor shape: {test_factors_norm.shape}")  # Expected: (1, 20, 6)
        
        ############# train using combined SHAP measures and test factor against binary test return 
        # Normalize SHAP values using the same mean and std as training factors
        shap_values_squeezed = np.squeeze(shap_values[0], axis=(0, -1))  # Shape: (60, 6)
        shap_values_normalized = (shap_values_squeezed - train_factors_mean.values) / train_factors_std.values
        
        # Concatenate normalized SHAP values with normalized test factors
        test_factors_np = test_factors_norm.values.astype('float32')  # Shape: (60, 6)
        augmented_features = np.concatenate([test_factors_np, shap_values_normalized], axis=1)  # Shape: (60, 12)
        
        # Convert test_returns_norm into binary (above/below mean)
        returns_mean = train_factors_mean.values
        test_returns_binary = (test_returns_norm > returns_mean).astype(int)

        # Train the Model Using Augmented Features
        
        d_model2 = augmented_features.shape[1]  # Number of features (6 original + 6 SHAP)
        num_heads = min(4, d_model2)
        while d_model2 % num_heads != 0:
            num_heads -= 1
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Create dataset and dataloader
        dataset = StockDatasetBinary(augmented_features, test_returns_norm, sequence_length)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        # Initialize the model for binary classification
        model = StockPredictAgentBinary(d_model=d_model2, num_heads=num_heads, sequence_length=sequence_length).to(device)
        
        # Optimizer and binary cross-entropy loss
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
        criterion = nn.BCELoss()  # Binary cross-entropy for up/down classification
        
        # Training loop
        def train_model(model, dataloader, optimizer, criterion, num_epochs, device):
            model.train()
            for epoch in range(num_epochs):
                total_loss = 0.0
                for factors, target in dataloader:
                    factors, target = factors.to(device), target.to(device).unsqueeze(1)  # Ensure correct shape
        
                    optimizer.zero_grad()
                    output = model(factors)
        
                    loss = criterion(output, target)
                    loss.backward()
                    optimizer.step()
        
                    total_loss += loss.item()
        
                avg_loss = total_loss / len(dataloader)
                print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
        
        # Train the model
        train_model(model, dataloader, optimizer, criterion, num_epochs, device)
        # Convert augmented test features to tensor
        augmented_features_tensor = torch.tensor(augmented_features, dtype=torch.float32).to(device)
        
        # Get model predictions
        model.eval()
        with torch.no_grad():
            test_predictions = model(augmented_features_tensor).cpu().numpy().flatten()
        
        # Convert probabilities to binary labels (0 = down, 1 = up)
        test_predictions_binary = (test_predictions > 0.5).astype(int)
        
        # Print predictions
        print(f"Predicted Stock Directions: {test_predictions_binary}")

        
        ########################## end of Shap test data training and use those model for prediction phase
        
        
        
        # ###### print SHAP RELATED METRICS
        # # shap_values_squeeze = shap_values[0].reshape(1,rolling_window_test,d_model)
        # 
        # # 1. Mean Absolute SHAP Values (Feature Importance)
        # mean_abs_shap_values = np.mean(np.abs(shap_values[0]), axis=(0, 1))  # Shape: (6,)
        # # print(f'mean_abs_shap_values: {mean_abs_shap_values}')
        # # 2. SHAP Value Variance (Variability of Feature Contributions)
        # shap_value_variance = np.var(shap_values[0], axis=(0, 1))  # Shape: (6,)
        # # print(f'shap_value_variance: {shap_value_variance}')
        # # 3. SHAP Interaction Values (Feature Interdependencies)
        # # interaction_values = shap.TreeExplainer(model).shap_interaction_values(test_factors_tensor)
        # 
        # # 4. Create DataFrame to Display Metrics
        # feature_importance_df = pd.DataFrame({
        #     'Date': test_factors.index[-1],
        #     'Feature': factors_raw_X.columns,
        #     'Mean_Abs_SHAP_Value': mean_abs_shap_values.flatten(),
        #     'SHAP_Value_Variance': shap_value_variance.flatten()
        #     # 'SHAP_Value_MEAN_DIV_VAR': mean_abs_shap_values.flatten() / shap_value_variance.flatten()
        #     
        # })
        # 
        # feature_importance_df_pivot = feature_importance_df.pivot(
        #         index='Date', 
        #         columns='Feature', 
        #         values=['Mean_Abs_SHAP_Value', 'SHAP_Value_Variance']
        #     ).reset_index()
        # # Sort by Feature Importance
        # # feature_importance_df = feature_importance_df.sort_values(by='Mean_Abs_SHAP_Value', ascending=False)
        # 
        # # Flatten MultiIndex columns for readability
        # feature_importance_df_pivot.columns = ['_'.join(col).strip() if col[1] else col[0] for col in feature_importance_df_pivot.columns.values]
        # 
        # 
        # 
        # print("Feature Importance Metrics (Mean SHAP & Variance):")
        # print(feature_importance_df_pivot.to_string(index=False))
        # # print(f'feature_importance_df: {feature_importance_df}')
        # 
        # # # end shap
        # ###### print SHAP RELATED METRICS
        
        
        
        future_predictions = future_predictions_norm * train_returns_std.item() + train_returns_mean.item()
        actual_test_returns = test_returns.values.flatten()
        actual_predict_returns = predict_returns.values.flatten()
        # print(f'test_predictions dim:{test_predictions.shape} actual_test_returns dim: {actual_test_returns.shape}')

        # Compute loss for test phase
        test_loss = np.mean((test_predictions - actual_test_returns) ** 2)  # MSE Loss
        all_test_losses.append(test_loss)
        print(f'test_loss: {test_loss}')
        # Store test results
        combined_test_result = {
            "Stock": stock,
            "Test Dates": (test_start, test_end),
            "Predicted Returns (Test Set)": test_predictions.tolist(),
            "Actual Returns (Test Set)": actual_test_returns.tolist(),
            "Loss": test_loss
        }
        
        # Add SHAP feature importance metrics to the test results
        combined_test_result.update(feature_importance_df_pivot.iloc[0].to_dict())
        
        # Append the combined results to the test_results list
        test_results.append(combined_test_result)


        # Store prediction results
        # prediction_results.append({
        #     "Stock": stock,
        #     "Prediction Dates": (predict_start, predict_end),
        #     "Predicted Returns": future_predictions.tolist(),
        #     "Actual Returns": actual_predict_returns.tolist()
        # })
        # Match dates with predictions and actual returns
        # date_range = pd.date_range(start=test_start, periods=len(actual_predict_returns), freq="D")
    
        for date, predicted, actual in zip(predict_returns.index, future_predictions, actual_predict_returns):
            prediction_results.append({
                "Stock": stock,
                "Date": date,
                "Predicted Return": predicted,
                "Actual Return": actual
            })
        print(f"Predictions for {stock} from {predict_start} to {predict_end} complete.")
        start += prediction_window
        
    # Print average test loss for this stock
    avg_test_loss = np.mean(all_test_losses)
    print(f"Average Test Loss for {stock}: {avg_test_loss:.6f}")
future_predictions_df = pd.DataFrame(prediction_results)  

future_predictions_df.to_csv(f"{input_dir}/predicted_vs_actual.csv", index=False)
test_results_df = pd.DataFrame(test_results)
test_results_df.to_csv(f"{input_dir}/test_result_output.csv", index=False)

In [ ]:
feature_importance_df


In [ ]:
import sys
from SVERL_icml_2023.portfolio_DRL.data_function import *
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader
from SVERL_icml_2023.portfolio_DRL.create_model import *
from sklearn.model_selection import TimeSeriesSplit
from SVERL_icml_2023.shapley import Shapley
import shap
sys.path.append("E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023")
# from portfolio_DRL.create_model import StockPredictWrapper


# Example usage
input_dir = "E:\XAI\RL_with_SHAP\pythonProject1\SVERL_icml_2023\portfolio_DRL"  # Use the specified directory
ticker_file = os.path.join(input_dir, "tickers.txt")  # Text file containing tickers, one per line
factor_file = os.path.join(input_dir, "FF_factors.csv")  # CSV file containing factor returns
start_date = "2020-01-01"
end_date = "2023-12-31"
output_csv = os.path.join(input_dir, "stock_prices.csv")
#
# try:
#     tickers = load_tickers_from_file(ticker_file)
#     price_data = download_stock_prices(tickers, start_date, end_date, output_csv)
#     daily_returns = calculate_daily_returns(price_data, return_type='arithmetic')
#     daily_returns.to_csv(os.path.join(input_dir, "daily_returns.csv"))
#     factor_returns = read_factor_returns(factor_file, daily_returns)
#     factor_returns.to_csv(os.path.join(input_dir, "aligned_factors.csv"))
#     print("Factor returns alignment completed.")
# except Exception as e:
#     print(f"An error occurred: {e}")

factors = pd.read_csv(f"{input_dir}/aligned_factors.csv", index_col=0, parse_dates=True)
returns_raw = pd.read_csv(f"{input_dir}/daily_returns.csv", index_col=0, parse_dates=True)


# Remove NaN values and align dates
factors = factors.dropna()
returns = returns_raw.dropna()

# Align dates between factors and returns
dates = factors.index.intersection(returns.index)
factors_raw = factors.loc[dates]
returns_raw = returns.loc[dates]
factors_raw = factors_raw.iloc[:, :-1]
# print(f"factors_raw = {factors_raw}")

Nday_diff = 60  # Define the future prediction horizon for shift days
returns_raw = returns_raw.iloc[:, :1]

# Remove NaN values and align dates
factors_raw = factors_raw.dropna()
returns_raw = returns_raw.dropna()
# factors_raw = pd.concat([factors_raw, returns_raw], axis=1)
# Align dates between factors_raw and returns_raw
dates = factors_raw.index.intersection(returns_raw.index)
factors_raw = factors_raw.loc[dates]
returns_raw = returns_raw.loc[dates]

# Shift returns to generate target variable (N-day future return)
returns_raw_Y = returns_raw.shift(-Nday_diff).dropna()

# Align all datasets with available dates in returns_raw_Y
dates_final = factors_raw.index.intersection(returns_raw_Y.index)
factors_raw = factors_raw.loc[dates_final]
returns_raw = returns_raw.loc[dates_final]
returns_raw_Y = returns_raw_Y.loc[dates_final]

N = 60
sequence_length = 60
batch_size = 16
num_epochs = 100
learning_rate = 0.001
rolling_window_train = 240  # Number of training samples
rolling_window_test = 60  # Number of testing samples
prediction_window = 60  # Number of prediction samples

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Record results
results = []
prediction_records = []
shapley_records = []
shapley_metrics_df = []
all_test_losses = []
test_results = []
prediction_results = []
combined_test_result = []

for stock in returns_raw.columns:
    print(f"Training for stock: {stock}")
    stock_returns = returns_raw_Y[[stock]]  # Select the stock column
    factors_raw_X = pd.concat([factors_raw, stock_returns], axis=1)
    # Rolling split: Train (M days), Test (N days), Predict (W days)
    total_samples = len(factors_raw_X)
    # Initialize the rolling split index
    start = 0

    while start + rolling_window_train + prediction_window + prediction_window <= total_samples:
        train_start = start
        train_end = start + rolling_window_train

        test_start = train_end
        test_end = test_start + rolling_window_test

        predict_start = test_end  # Predictions start right after training
        predict_end = predict_start + prediction_window

        # Extract the actual date indices
        train_dates = (factors_raw_X.iloc[train_start].name, factors_raw_X.iloc[train_end - 1].name)
        test_dates = (factors_raw_X.iloc[test_start].name, factors_raw_X.iloc[test_end - 1].name)
        predict_dates = (factors_raw_X.iloc[predict_start].name, factors_raw_X.iloc[predict_end - 1].name)

        # Print the selected date ranges
        print(f"Training dates: {train_dates}")
        print(f"Testing dates: {test_dates}")
        print(f"Prediction dates: {predict_dates}")

        # # Split data into training, testing, and prediction sets
        train_factors = factors_raw_X.iloc[train_start:train_end]
        train_returns = stock_returns.iloc[train_start:train_end]
        test_factors = factors_raw_X.iloc[test_start:test_end]
        test_returns = stock_returns.iloc[test_start:test_end]
        predict_factors = factors_raw_X.iloc[predict_start:predict_end]
        predict_returns = stock_returns.iloc[predict_start:predict_end]
        #
        # # Normalize train and test data
        train_factors_mean, train_factors_std = train_factors.mean(), train_factors.std()
        train_returns_mean, train_returns_std = train_returns.mean(), train_returns.std()

        # Avoid division by zero
        train_factors_std.replace(0, 1e-8, inplace=True)
        train_returns_std.replace(0, 1e-8, inplace=True)

        train_factors_norm = (train_factors - train_factors_mean) / train_factors_std
        train_returns_norm = (train_returns - train_returns_mean) / train_returns_std
        test_factors_norm = (test_factors - train_factors_mean) / train_factors_std
        test_returns_norm = (test_returns - train_returns_mean) / train_returns_std
        predict_factors_norm = (predict_factors - train_factors_mean) / train_factors_std
        predict_returns_norm = (predict_returns - train_returns_mean) / train_returns_std
        print(f'train_factors_norm dim: {train_factors_norm.shape}')
        print(f'train_returns_norm dim: {train_returns_norm.shape}')
        dataset = StockDataset(train_factors_norm, train_returns_norm, sequence_length, N=N)

        sample_x, sample_y = dataset[0]

        print(f"🚀 Step 1: Sample X Raw Shape from Dataset: {sample_x.shape}")  # Expected: (60, 6)
        print(f"🚀 Step 1: Sample Y Raw Shape from Dataset: {sample_y.shape}")  # Expected: (60, 1)

        # Explicitly check for extra dimensions
        if sample_x.shape[0] == 1 and len(sample_x.shape) == 3:
            print("🔥 Warning: Extra dimension detected in Sample X!")

        # # Initialize dataset and dataloader
        dataset = StockDataset(train_factors_norm, train_returns_norm, sequence_length, N=N)

        sample_x, sample_y = dataset[0]

        print(f"Fixed Sample X Shape: {sample_x.shape}")  # Expected: (60, 6)
        print(f"Fixed Sample Y Shape: {sample_y.shape}")  # Expected: (60, 1)

        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        #
        # # Model initialization
        d_model = train_factors.shape[1]
        print(f'd_model: {d_model}')

        num_heads = min(4, d_model)  # Ensure num_heads does not exceed d_model and d_model is divisible by num_heads
        while d_model % num_heads != 0:
            num_heads -= 1
        # print(f"d_model: {d_model} num_heads: {num_heads} sequence_length: {sequence_length} and train factor return dim: {train_factors.shape[1]} {train_returns.shape[1]}")
        model = StockPredictAgent(d_model=d_model, num_heads=num_heads, sequence_length=sequence_length, N=N).to(device)
        #
        # # Optimizer and loss function
        optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
        criterion = nn.SmoothL1Loss(beta=1.0)  # Huber Loss for stability
        #
        # # Training the model
        # print(f"Training window: {train_start} to {train_end}")
        # print(f"Expected FC Layer Input Shape: {sequence_length * d_model}")
        # print(f"Flattened Tensor Shape: {batch_size, sequence_length * d_model}")

        train_model_with_logging(model, dataloader, optimizer, criterion, scheduler, num_epochs, device)

        # Convert to PyTorch tensors
        test_factors_tensor = torch.tensor(test_factors_norm.values, dtype=torch.float32).to(device).unsqueeze(0)
        predict_factors_tensor = torch.tensor(predict_factors_norm.values, dtype=torch.float32).to(device).unsqueeze(0)
        print(f'test_factors_tensor: {test_factors_tensor.shape}')
        # print(f'test_factors_tensor: {test_factors_tensor.shape}')
        # Run inference on test data
        model.eval()
        with torch.no_grad():
            test_predictions_norm = model(test_factors_tensor).cpu().numpy().flatten()
            future_predictions_norm = model(predict_factors_tensor).cpu().numpy().flatten()

            # test_output = model(torch.tensor(test_factors_np, dtype=torch.float32).to(device))
            # print(f"Model output shape: {test_output.shape}")  # Expected: (1, 20) for 20 future returns
        # print(f'test_factors_tensor: {test_factors_tensor.shape}')
        # Revert normalization for test and predicted returns
        test_predictions = test_predictions_norm * train_returns_std.item() + train_returns_mean.item()

        #
        # # start SHAP
        # Prepare your input: (1 sample, 20 days, 6 features)
        # ######################## WRAP THIS AS FUNCTION
        # test_factors_np = test_factors_norm.values.astype('float32')
        # test_factors_np = test_factors_np.reshape(1, rolling_window_test,
        #                                           d_model)  # Reshaping for 1 sample with 20 days and 7 features
        # # print(f'test_factors_np dim: {test_factors_np.shape}')
        # #
        # # # Use a subset of the training data as the background dataset
        # background_data = torch.tensor(train_factors_norm.values.astype('float32')).to(device)
        # background_data = background_data.reshape(-1, rolling_window_test, d_model)  # Ensure same shape as model input
        # # print(f'background_data dim: {background_data.shape}')
        # # Ensure background_data is a tensor and avoid redundant torch.tensor() calls
        # if not isinstance(background_data, torch.Tensor):
        #     background_data = torch.tensor(background_data, dtype=torch.float32).to(device)
        # else:
        #     background_data = background_data.to(device)  # Just move to device without re-wrapping
        #
        # # Initialize SHAP GradientExplainer
        # explainer = shap.GradientExplainer(model, background_data)
        #
        # # Compute SHAP values for the test input
        # shap_values = explainer.shap_values(torch.tensor(test_factors_np, dtype=torch.float32).to(device),
        #                                     ranked_outputs=1)
        # # Aggregate SHAP values across outputs and time
        # # Verify SHAP values shape
        # # print(f"SHAP values shape: {shap_values[0].shape}")  # Expected: (1, 20, 6)
        # # print(f"test factor shape: {test_factors_norm.shape}")  # Expected: (1, 20, 6)
        #
        # ############# train using combined SHAP measures and test factor against binary test return
        # # Normalize SHAP values using the same mean and std as training factors
        # shap_values_squeezed = np.squeeze(shap_values[0], axis=(0, -1))  # Shape: (60, 6)
        # shap_values_normalized = (shap_values_squeezed - train_factors_mean.values) / train_factors_std.values
        #
        # # Concatenate normalized SHAP values with normalized test factors
        # test_factors_np = test_factors_norm.values.astype('float32')  # Shape: (60, 6)
        # augmented_features = np.concatenate([test_factors_np, shap_values_normalized], axis=1)  # Shape: (60, 12)
        # ########## END OF WRAPPING FUNCTION
        # Generate augmented features
        augmented_test_features = generate_augmented_features(
            test_factors_norm, train_factors_norm, train_factors_mean, train_factors_std,
            model, rolling_window_test, device
        )

        # Convert to PyTorch tensor if needed
        augmented_features_tensor = torch.tensor(augmented_test_features, dtype=torch.float32).to(device)

        print(f"Final Augmented Tensor Shape: {augmented_features_tensor.shape}")  # Expected: (rolling_window_test, d_model * 2)

        # Convert test_returns_norm into binary classification
        cumulative_return = np.prod(test_returns_norm + 1) - 1  # Aggregate 60-day return
        test_returns_binary = (cumulative_return > train_returns_mean).astype(int)  # 1 if positive, 0 if negative

        print(f"Cumulative Return over 60 days: {cumulative_return}")
        print(f"Binary Target (Up=1, Down=0): {test_returns_binary}")

        # Train the Model Using Augmented Features
        # sequence_length = 20
        # batch_size = 32
        # num_epochs = 25
        # learning_rate = 0.001
        d_model2 = augmented_test_features.shape[1]  # Number of features (6 original + 6 SHAP)
        num_heads = min(4, d_model2)
        while d_model2 % num_heads != 0:
            num_heads -= 1

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Length of augmented_test_features: {augmented_test_features.shape}")
        print(f"Length of test_returns_norm: {test_returns_norm.shape}")
        print(f"Length of sequence: {sequence_length}")

        sequence_length2 = min(10, len(augmented_test_features))  # Ensure it's smaller than available data

        # Create dataset and dataloader for training
        dataset2 = StockDatasetBinary(augmented_test_features, test_returns_norm, sequence_length2)
        dataloader2 = DataLoader(dataset2, batch_size=batch_size, shuffle=True)

        for batch_x, batch_y in dataloader2:
            print(f"Batch X Shape: {batch_x.shape}")  # Expected: (batch_size, sequence_length, num_features)
            print(f"Batch Y Shape: {batch_y.shape}")  # Expected: (batch_size, N, num_features)
            break  # Only print for the first batch

        # Initialize the binary classification model
        model_binary = StockPredictAgentBinary(d_model=d_model2, num_heads=num_heads,
                                               sequence_length=sequence_length2).to(device)

        # Define optimizer and loss function
        optimizer = optim.Adam(model_binary.parameters(), lr=learning_rate, weight_decay=1e-5)
        criterion = nn.BCELoss()  # Binary cross-entropy loss

        # Train the model
        train_model(model_binary, dataloader2, optimizer, criterion, num_epochs, device)
        # Convert augmented test features to tensor

        augmented_predict_features = generate_augmented_features(
            predict_factors_norm, train_factors_norm, train_factors_mean, train_factors_std,
            model, prediction_window, device
        )
        augmented_features_tensor_predict = torch.tensor(augmented_predict_features, dtype=torch.float32).to(device)
        # augmented_features_tensor_predict = torch.tensor(augmented_predict_features, dtype=torch.float32).to(device).unsqueeze(0)
        augmented_features_tensor_predict = augmented_features_tensor_predict[-10:]  # Select the last 10 time steps
        augmented_features_tensor_predict = augmented_features_tensor_predict.unsqueeze(0)  # Add batch dimension

        # Ensure input has (batch_size=1, sequence_length=60, d_model=12)
        # augmented_features_tensor_predict = augmented_features_tensor_predict.view(1, 60, 12)  # Reshape explicitly

        print(f"Fixed Shape: {augmented_features_tensor_predict.shape}")  # Should be


        # Get model predictions
        model_binary.eval()
        with torch.no_grad():
            return_predictions_prob = model_binary(augmented_features_tensor_predict).cpu().numpy().flatten()
            # return_test_prob = model_binary(augmented_features_tensor).cpu().numpy().flatten()


        # Convert probabilities to binary labels (0 = down, 1 = up)
        # Convert probability outputs to binary labels (0 = down, 1 = up)
        print(f'return_predictions_prob: {return_predictions_prob}')
        return_predictions_binary = (return_predictions_prob > 0.5).astype(int)
        #
        # # Display predictions
        # print(f"Predicted Binary Stock Returns: {return_predictions_binary}")

        # Print predictions
        # print(f"Predicted Stock Directions: {return_predictions_binary}")

        # ###### print SHAP RELATED METRICS
        # # shap_values_squeeze = shap_values[0].reshape(1,rolling_window_test,d_model)
        #
        # # 1. Mean Absolute SHAP Values (Feature Importance)
        # mean_abs_shap_values = np.mean(np.abs(shap_values[0]), axis=(0, 1))  # Shape: (6,)
        # # print(f'mean_abs_shap_values: {mean_abs_shap_values}')
        # # 2. SHAP Value Variance (Variability of Feature Contributions)
        # shap_value_variance = np.var(shap_values[0], axis=(0, 1))  # Shape: (6,)
        # # print(f'shap_value_variance: {shap_value_variance}')
        # # 3. SHAP Interaction Values (Feature Interdependencies)
        # # interaction_values = shap.TreeExplainer(model).shap_interaction_values(test_factors_tensor)
        #
        # # 4. Create DataFrame to Display Metrics
        # feature_importance_df = pd.DataFrame({
        #     'Date': test_factors.index[-1],
        #     'Feature': factors_raw_X.columns,
        #     'Mean_Abs_SHAP_Value': mean_abs_shap_values.flatten(),
        #     'SHAP_Value_Variance': shap_value_variance.flatten()
        #     # 'SHAP_Value_MEAN_DIV_VAR': mean_abs_shap_values.flatten() / shap_value_variance.flatten()
        #
        # })
        #
        # feature_importance_df_pivot = feature_importance_df.pivot(
        #         index='Date',
        #         columns='Feature',
        #         values=['Mean_Abs_SHAP_Value', 'SHAP_Value_Variance']
        #     ).reset_index()
        # # Sort by Feature Importance
        # # feature_importance_df = feature_importance_df.sort_values(by='Mean_Abs_SHAP_Value', ascending=False)
        #
        # # Flatten MultiIndex columns for readability
        # feature_importance_df_pivot.columns = ['_'.join(col).strip() if col[1] else col[0] for col in feature_importance_df_pivot.columns.values]
        #
        #
        #
        # print("Feature Importance Metrics (Mean SHAP & Variance):")
        # print(feature_importance_df_pivot.to_string(index=False))
        # # print(f'feature_importance_df: {feature_importance_df}')
        #
        # # # end shap
        # ###### print SHAP RELATED METRICS

        future_predictions = future_predictions_norm * train_returns_std.item() + train_returns_mean.item()
        actual_test_returns = test_returns.values.flatten()
        actual_predict_returns = predict_returns.values.flatten()
        # print(f'test_predictions dim:{test_predictions.shape} actual_test_returns dim: {actual_test_returns.shape}')

        # Compute loss for test phase
        test_loss = np.mean((test_predictions - actual_test_returns) ** 2)  # MSE Loss
        all_test_losses.append(test_loss)
        print(f'test_loss: {test_loss}')
        # Store test results
        combined_test_result = {
            "Stock": stock,
            "Test Dates": (test_start, test_end),
            "Predicted Returns (Test Set)": test_predictions.tolist(),
            "Actual Returns (Test Set)": actual_test_returns.tolist(),
            "Loss": test_loss
        }

        # Add SHAP feature importance metrics to the test results
        # combined_test_result.update(feature_importance_df_pivot.iloc[0].to_dict())

        # Append the combined results to the test_results list
        test_results.append(combined_test_result)

        # Store prediction results
        # prediction_results.append({
        #     "Stock": stock,
        #     "Prediction Dates": (predict_start, predict_end),
        #     "Predicted Returns": future_predictions.tolist(),
        #     "Actual Returns": actual_predict_returns.tolist()
        # })
        # Match dates with predictions and actual returns
        # date_range = pd.date_range(start=test_start, periods=len(actual_predict_returns), freq="D")

        for date, predicted, actual in zip(predict_returns.index, future_predictions, actual_predict_returns):
            prediction_results.append({
                "Stock": stock,
                "Date": date,
                "Predicted Return": predicted,
                "Actual Return": actual
            })
        print(f"Predictions for {stock} from {predict_start} to {predict_end} complete.")
        start += prediction_window

    # Print average test loss for this stock
    avg_test_loss = np.mean(all_test_losses)
    print(f"Average Test Loss for {stock}: {avg_test_loss:.6f}")
future_predictions_df = pd.DataFrame(prediction_results)

future_predictions_df.to_csv(f"{input_dir}/predicted_vs_actual.csv", index=False)
test_results_df = pd.DataFrame(test_results)
test_results_df.to_csv(f"{input_dir}/test_result_output.csv", index=False)

In [ ]:
future_predictions_df = pd.DataFrame(prediction_results)  

future_predictions_df.to_csv(f"{input_dir}/predicted_vs_actual.csv", index=False)
test_results_df = pd.DataFrame(test_results)
test_results_df.to_csv(f"{input_dir}/test_result_output.csv", index=False)

In [ ]:
    test_results_df = pd.DataFrame(test_results)
    test_results_df.to_csv(f"{input_dir}/test_result_output.csv", index=False)

In [ ]:
background_data.shape[0]

In [ ]:
predict_returns.index

In [ ]:
all_test_losses

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

class RollingStockFramework:
    def __init__(self, model, factors, returns, rolling_window_train, rolling_window_test, M, N, batch_size, num_epochs, learning_rate, device):
        self.model = model.to(device)
        self.factors = factors
        self.returns = returns
        self.rolling_window_train = rolling_window_train
        self.rolling_window_test = rolling_window_test
        self.M = M  # Mini-batch sequence length
        self.N = N  # Prediction horizon
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.device = device

    def create_rolling_windows(self, factors, returns, window_size):
        X, Y = [], []
        num_samples = window_size - self.M - self.N
        
        if num_samples <= 0:
            print(f"⚠️ Warning: Insufficient data for rolling windows! window_size={window_size}, M={self.M}, N={self.N}")
            return torch.empty((0, self.M, factors.shape[1] + 1)), torch.empty((0, self.N))  # Return correctly shaped empty tensors

        for start in range(num_samples):
            factor_values = torch.tensor(factors.iloc[start:start+self.M].values, dtype=torch.float32)
            return_values = torch.tensor(returns.iloc[start:start+self.M].values, dtype=torch.float32)
            
            # print(f"factor_values shape: {factor_values.shape}, return_values shape: {return_values.shape}")
            X.append(torch.cat([factor_values, return_values], dim=-1))
            Y.append(torch.tensor(returns.iloc[start+self.M:start+self.M+self.N].values, dtype=torch.float32))

        X_stack = torch.stack(X) if X else torch.empty((0, self.M, factors.shape[1] + 1))
        Y_stack = torch.stack(Y) if Y else torch.empty((0, self.N))
        
        print(f"Final X_train shape: {X_stack.shape}, Y_train shape: {Y_stack.shape}")
        return X_stack, Y_stack

    def train(self, train_factors, train_returns):
        X_train, Y_train = self.create_rolling_windows(train_factors, train_returns, self.rolling_window_train)
        if X_train.shape[0] == 0:
            print("⚠️ Skipping training due to insufficient data.")
            return
        dataset = torch.utils.data.TensorDataset(X_train, Y_train)
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

        self.model.train()
        for epoch in range(self.num_epochs):
            total_loss = 0.0
            for factors_batch, target_batch in dataloader:
                factors_batch, target_batch = factors_batch.to(self.device), target_batch.to(self.device)
                
                optimizer.zero_grad()
                predictions = self.model(factors_batch)
                target_batch = target_batch.squeeze(-1) if predictions.shape[-1] != target_batch.shape[-1] else target_batch
                loss = criterion(predictions, target_batch)
                total_loss += loss.item()
                
                loss.backward()
                optimizer.step()

            print(f"Epoch {epoch+1}/{self.num_epochs}, Loss: {total_loss / len(dataloader):.4f}")

    def test(self, test_factors, test_returns):
        X_test, Y_test = self.create_rolling_windows(test_factors, test_returns, self.rolling_window_test)
        if X_test.shape[0] == 0:
            print("⚠️ Skipping testing due to insufficient data.")
            return float('nan')
        dataset = torch.utils.data.TensorDataset(X_test, Y_test)
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False)

        criterion = nn.MSELoss()
        self.model.eval()

        total_test_loss = 0.0
        with torch.no_grad():
            for factors_batch, target_batch in dataloader:
                factors_batch, target_batch = factors_batch.to(self.device), target_batch.to(self.device)
                predictions = self.model(factors_batch)
                target_batch = target_batch.squeeze(-1) if predictions.shape[-1] != target_batch.shape[-1] else target_batch
                loss = criterion(predictions, target_batch)
                total_test_loss += loss.item()

        avg_test_loss = total_test_loss / len(dataloader) if len(dataloader) > 0 else float('nan')
        print(f"Test Loss: {avg_test_loss:.4f}")
        return avg_test_loss


    def predict(self, predict_factors, predict_returns):
        X_predict, _ = self.create_rolling_windows(predict_factors, predict_returns, self.rolling_window_test)
        dataset = torch.utils.data.TensorDataset(X_predict, torch.zeros(X_predict.shape[0]))
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False)

        predictions_list = []
        self.model.eval()

        with torch.no_grad():
            for factors_batch, _ in dataloader:
                factors_batch = factors_batch.to(self.device)
                predictions = self.model(factors_batch)
                predictions_list.append(predictions.cpu().numpy())

        predictions = torch.cat([torch.tensor(p) for p in predictions_list], dim=0).numpy()
        return predictions

    def run(self):
        results = []
        for start in range(0, len(self.factors) - self.rolling_window_train - self.rolling_window_test, self.N):
            print(f"\n Rolling window starting at index {start}")

            train_factors = self.factors.iloc[start:start+self.rolling_window_train]
            train_returns = self.returns.iloc[start:start+self.rolling_window_train]
            test_factors = self.factors.iloc[start+self.rolling_window_train:start+self.rolling_window_train+self.rolling_window_test]
            test_returns = self.returns.iloc[start+self.rolling_window_train:start+self.rolling_window_train+self.rolling_window_test]
            predict_factors = self.factors.iloc[-self.rolling_window_test:]
            predict_returns = self.returns.iloc[-self.rolling_window_test:]

            print(" Training Model...")
            self.train(train_factors, train_returns)

            print("\n Evaluating Model...")
            test_loss = self.test(test_factors, test_returns)

            print("\n Predicting Future Returns...")
            predictions = self.predict(predict_factors, predict_returns)

            results.append({
                "train_window": (start, start + self.rolling_window_train),
                "test_window": (start + self.rolling_window_train, start + self.rolling_window_train + self.rolling_window_test),
                "predict_window": (len(self.factors) - self.rolling_window_test, len(self.factors)),
                "test_loss": test_loss,
                "predictions": predictions
            })

        return self.model, results


In [ ]:
M = 60  # Define the mini-batch past sequence length
N = 20  # Define the future prediction horizon
returns_raw = returns_raw.iloc[:, :1]

# Remove NaN values and align dates
factors_raw = factors_raw.dropna()
returns_raw = returns_raw.dropna()

# Align dates between factors_raw and returns_raw
dates = factors_raw.index.intersection(returns_raw.index)
factors_raw = factors_raw.loc[dates]
returns_raw = returns_raw.loc[dates]

for stock in returns_raw.columns:
    print(f"Training for stock: {stock}")
    stock_returns = returns_raw[[stock]]  # Select the stock column
    
    # Ensure d_model matches 6 factors + 1 stock return
    d_model = factors_raw.shape[1] + 1  # Fix to match input shape
    # print(f'd_model dim: {d_model}')
    num_heads = min(4, d_model)  # Ensure num_heads does not exceed d_model
    while d_model % num_heads != 0:
        num_heads -= 1
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Initialize the model before passing into the pipeline
    model = StockPredictAgent(d_model=d_model, num_heads=num_heads, sequence_length=M, N=N).to(device)  # NEW
    
    # Now you can safely run the pipeline
    pipeline = RollingStockFramework(
        model, factors_raw, stock_returns, rolling_window_train=240, rolling_window_test=120,
        M=M, N=N, batch_size=32, num_epochs=50, learning_rate=0.001, device=device
    )
    
    trained_model, rolling_results = pipeline.run()


In [ ]:
# Select the first N columns of returns_raw
M = 60  # Define the mini-batch past sequence length
N = 20  # Define the future prediction horizon
returns_raw = returns_raw.iloc[:, :1]

# Remove NaN values and align dates
factors_raw = factors_raw.dropna()
returns_raw = returns_raw.dropna()

# Align dates between factors_raw and returns_raw
dates = factors_raw.index.intersection(returns_raw.index)
factors_raw = factors_raw.loc[dates]
returns_raw = returns_raw.loc[dates]
# print(f'factors_raw {factors_raw.shape}')

sequence_length = 20
batch_size = 20
num_epochs = 25
learning_rate = 0.001
rolling_window_train = 120  # Number of training samples
rolling_window_test = 60   # Number of testing samples
prediction_window = 60     # Number of prediction samples

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Record results
results = []
prediction_records = []
shapley_records = []
shapley_metrics_df = []

# Iterate over each stock in returns_raw
for stock in returns_raw.columns:
    print(f"Training for stock: {stock}")
    stock_returns = returns_raw[[stock]]  # Select the stock column

    # Rolling split: Train (M days), Test (N days), Predict (W days)
    total_samples = len(factors_raw)
    for start in range(0, total_samples - rolling_window_train - rolling_window_test - prediction_window + 1):
        train_start_X = start
        train_end_X = start + rolling_window_train
        train_start_Y = train_end_X
        train_end_Y = train_start_Y + N
        
        test_start_X = train_end_X
        test_end_X = train_end_X + rolling_window_test
        test_start_Y = test_end_X
        test_end_Y = test_start_Y + N
        
        predict_start_X = test_end_X
        predict_end_X = test_end_X + prediction_window
        test_start_Y = predict_end_X
        test_end_Y = predict_end_X + N

        # Print start and end dates for training, testing, and prediction
        train_dates = (factors_raw.iloc[train_start_X].name, factors_raw.iloc[train_end_X - 1].name)
        test_dates = (factors_raw.iloc[test_start].name, factors_raw.iloc[test_end - 1].name)
        predict_dates = (factors_raw.iloc[predict_start].name, factors_raw.iloc[predict_end - 1].name)

        print(f"Training dates: {train_dates}")
        print(f"Testing dates: {test_dates}")
        print(f"Prediction dates: {predict_dates}")

        # Split data into training, testing, and prediction sets
        train_factors = factors_raw.iloc[train_start:train_end]
        train_returns = stock_returns.iloc[train_start:train_end]
        test_factors = factors_raw.iloc[test_start:test_end]
        test_returns = stock_returns.iloc[test_start:test_end]
        predict_factors = factors_raw.iloc[predict_start:predict_end]
        predict_returns = stock_returns.iloc[predict_start:predict_end]

        # Normalize train and test data
        train_factors_mean, train_factors_std = train_factors.mean(), train_factors.std()
        train_returns_mean, train_returns_std = train_returns.mean(), train_returns.std()

        train_factors = (train_factors - train_factors_mean) / train_factors_std
        train_returns = (train_returns - train_returns_mean) / train_returns_std
        test_factors = (test_factors - train_factors_mean) / train_factors_std
        test_returns = (test_returns - train_returns_mean) / train_returns_std

        # Initialize dataset and dataloader
        dataset = StockDataset(train_factors, train_returns, sequence_length)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        # Model initialization
        d_model = train_factors.shape[1] 
        num_heads = min(4, d_model)  # Ensure num_heads does not exceed d_model and d_model is divisible by num_heads
        while d_model % num_heads != 0:
            num_heads -= 1
        # print(f"d_model: {d_model} num_heads: {num_heads} sequence_length: {sequence_length} and train factor return dim: {train_factors.shape[1]} {train_returns.shape[1]}")
        model = StockPredictAgent(d_model=d_model, num_heads=num_heads, sequence_length=M, N=N).to(device)

        # Optimizer and loss function
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.MSELoss()

        # Training the model
        print(f"Training window: {train_start} to {train_end}")
        train_model_with_logging(model, dataloader, optimizer, criterion, num_epochs, device)

In [ ]:
        
# Select the first N columns of returns_raw
N = 2  # Number of columns to select
returns_raw = returns_raw.iloc[:, :N]

# Remove NaN values and align dates
factors_raw = factors_raw.dropna()
returns_raw = returns_raw.dropna()

# Align dates between factors_raw and returns_raw
dates = factors_raw.index.intersection(returns_raw.index)
factors_raw = factors_raw.loc[dates]
returns_raw = returns_raw.loc[dates]

sequence_length = 20
batch_size = 20
num_epochs = 25
learning_rate = 0.001
rolling_window_train = 120  # Number of training samples
rolling_window_test = 60   # Number of testing samples
prediction_window = 60     # Number of prediction samples

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Record results
results = []
prediction_records = []
shapley_records = []
shapley_metrics_df = []

# Iterate over each stock in returns_raw
for stock in returns_raw.columns:
    print(f"Training for stock: {stock}")
    stock_returns = returns_raw[[stock]]  # Select the stock column

    # Rolling split: Train (M days), Test (N days), Predict (W days)
    total_samples = len(factors_raw)
    for start in range(0, total_samples - rolling_window_train - rolling_window_test - prediction_window + 1):
        train_start = start
        train_end = start + rolling_window_train
        test_start = train_end
        test_end = train_end + rolling_window_test
        predict_start = test_end
        predict_end = test_end + prediction_window

        # Print start and end dates for training, testing, and prediction
        train_dates = (factors_raw.iloc[train_start].name, factors_raw.iloc[train_end - 1].name)
        test_dates = (factors_raw.iloc[test_start].name, factors_raw.iloc[test_end - 1].name)
        predict_dates = (factors_raw.iloc[predict_start].name, factors_raw.iloc[predict_end - 1].name)

        print(f"Training dates: {train_dates}")
        print(f"Testing dates: {test_dates}")
        print(f"Prediction dates: {predict_dates}")

        # Split data into training, testing, and prediction sets
        train_factors = factors_raw.iloc[train_start:train_end]
        train_returns = stock_returns.iloc[train_start:train_end]
        test_factors = factors_raw.iloc[test_start:test_end]
        test_returns = stock_returns.iloc[test_start:test_end]
        predict_factors = factors_raw.iloc[predict_start:predict_end]
        predict_returns = stock_returns.iloc[predict_start:predict_end]

        # Normalize train and test data
        train_factors_mean, train_factors_std = train_factors.mean(), train_factors.std()
        train_returns_mean, train_returns_std = train_returns.mean(), train_returns.std()

        train_factors = (train_factors - train_factors_mean) / train_factors_std
        train_returns = (train_returns - train_returns_mean) / train_returns_std
        test_factors = (test_factors - train_factors_mean) / train_factors_std
        test_returns = (test_returns - train_returns_mean) / train_returns_std

        # Initialize dataset and dataloader
        dataset = StockDataset(train_factors, train_returns, sequence_length)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        # Model initialization
        d_model = train_factors.shape[1] 
        num_heads = min(4, d_model)  # Ensure num_heads does not exceed d_model and d_model is divisible by num_heads
        while d_model % num_heads != 0:
            num_heads -= 1
        # print(f"d_model: {d_model} num_heads: {num_heads} sequence_length: {sequence_length} and train factor return dim: {train_factors.shape[1]} {train_returns.shape[1]}")
        model = StockPredictAgent(d_model=d_model, num_heads=num_heads, sequence_length=sequence_length).to(device)

        # Optimizer and loss function
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.MSELoss()

        # Training the model
        print(f"Training window: {train_start} to {train_end}")
        train_model_with_logging(model, dataloader, optimizer, criterion, num_epochs, device)

        # Testing phase
        if len(test_factors) >= sequence_length:
            test_dataset = StockDataset(test_factors, test_returns, sequence_length)
            test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
            # Wrap the model for SHAP
            model_wrapper = StockPredictWrapper(model, device)

            # SHAP Explainer Initialization
            explainer = shap.Explainer(model_wrapper, test_factors.to_numpy())
            model.eval()
            # shap = Shapley(test_factors.to_numpy())  # Initialize Shapley calculation
            with torch.no_grad():
                total_test_loss = 0.0
                for factors, target in test_dataloader:
                    factors, target = factors.to(device),target.to(device)
                    predictions = model(factors)
                    loss = criterion(predictions, target)
                    total_test_loss += loss.item()
                    
                    print(f"SHAP input factors shape: {test_factors.shape}")
                    print(f"SHAP input returns shape: {test_returns.shape}")

        
                    # # Calculate Shapley values for each factor
                    # shap_values = explainer(test_factors.to_numpy())
                    # 
                    # # Record Shapley metrics
                    # shap_metrics = calculate_shapley_metrics(shap_values, test_factors)
                    # shapley_records.append(shap_metrics["shapley_df"])
                    # 
                    # # Append metrics to records
                    # metrics_df = pd.DataFrame({
                    #     "date": test_factors.index,
                    #     "mean_abs_shap": shap_metrics["mean_abs_shap"],
                    #     "shapley_variance": shap_metrics["shapley_variance"],
                    # })
                    # metrics_df["top_factors_per_sample"] = shap_metrics["top_factors_per_sample"].apply(lambda x: ", ".join(x))
                    # shapley_metrics_df.append(metrics_df)

                avg_test_loss = total_test_loss / len(test_dataloader)
                print(f"Test Loss: {avg_test_loss:.4f}")
        else:
            print(f"Skipping testing phase for stock {stock} due to insufficient data for sequence length.")

        # Prediction phase
        if len(predict_factors) >= sequence_length:
            predict_factors = (predict_factors - train_factors_mean) / train_factors_std
            predict_returns = (predict_returns - train_returns_mean) / train_returns_std

            predict_dataset = StockDataset(predict_factors, predict_returns, sequence_length)
            predict_dataloader = DataLoader(predict_dataset, batch_size=batch_size, shuffle=False)

            predicted_returns = []
            actual_returns = []

            with torch.no_grad():
                for factors, target in predict_dataloader:
                    factors, target = factors.to(device),  target.to(device)
                    predictions = model(factors)
                    predicted_returns.append(predictions.cpu().numpy())
                    actual_returns.append(target.cpu().numpy())

            # Revert normalization for predicted and actual returns
            # predicted_returns = np.concatenate(predicted_returns) * train_returns_std.values[0] + train_returns_mean.values[0]
            # actual_returns = np.concatenate(actual_returns) * train_returns_std.values[0] + train_returns_mean.values[0]

            predicted_returns = np.concatenate(predicted_returns, axis=0).flatten() * train_returns_std.values[0] + train_returns_mean.values[0]
            actual_returns = np.concatenate(actual_returns, axis=0).flatten() * train_returns_std.values[0] + train_returns_mean.values[0]

            # Combine dates with returns
            prediction_data = pd.DataFrame({
                "date": predict_factors.index[-len(predicted_returns):],
                "predicted_returns": predicted_returns,
                "actual_returns": actual_returns
            })
            prediction_records.append(prediction_data)

            # Save results for this window
            results.append({
                "stock": stock,
                "train_window": (train_start, train_end),
                "test_window": (test_start, test_end),
                "predict_window": (predict_start, predict_end),
                "predicted_returns": predicted_returns,
                "actual_returns": actual_returns,
                "test_loss": avg_test_loss
            })

# Save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv(f"{input_dir}/model_results.csv", index=False)

# Save combined predicted and actual returns
combined_predictions = pd.concat(prediction_records, ignore_index=True)
combined_predictions.to_csv(f"{input_dir}/predicted_vs_actual.csv", index=False)

# Save Shapley time series
if shapley_records:
    combined_shapley = pd.concat(shapley_records, ignore_index=False)
    combined_shapley.to_csv(f"{input_dir}/shapley_time_series.csv")

# Save Shapley metrics as a combined CSV
if shapley_metrics_df:
    combined_metrics_df = pd.concat(shapley_metrics_df, ignore_index=True)
    combined_metrics_df.to_csv(f"{input_dir}/shapley_metrics.csv", index=False)

print("All stocks training, testing, and prediction completed.")


In [ ]:
returns_raw.columns[1]

In [ ]:
# Select the first N columns of returns_raw
N = 2  # Number of columns to select
returns_raw = returns_raw.iloc[:, :N]

# Remove NaN values and align dates
factors_raw = factors_raw.dropna()
returns_raw = returns_raw.dropna()

# Align dates between factors_raw and returns_raw
dates = factors_raw.index.intersection(returns_raw.index)
factors_raw = factors_raw.loc[dates]
returns_raw = returns_raw.loc[dates]

sequence_length = 20
batch_size = 20
num_epochs = 25
learning_rate = 0.001
rolling_window_train = 120  # Number of training samples
rolling_window_test = 60   # Number of testing samples
prediction_window = 60     # Number of prediction samples

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Record results
results = []
prediction_records = []
shapley_records = []
shapley_metrics_df = []

# Iterate over each stock in returns_raw
stock=returns_raw.columns[0]
print(f"Training for stock: {stock}")
stock_returns = returns_raw[[stock]]  # Select the stock column

# Rolling split: Train (M days), Test (N days), Predict (W days)
total_samples = len(factors_raw)
start= 0
train_start = start
train_end = start + rolling_window_train
test_start = train_end
test_end = train_end + rolling_window_test
predict_start = test_end
predict_end = test_end + prediction_window

# Print start and end dates for training, testing, and prediction
train_dates = (factors_raw.iloc[train_start].name, factors_raw.iloc[train_end - 1].name)
test_dates = (factors_raw.iloc[test_start].name, factors_raw.iloc[test_end - 1].name)
predict_dates = (factors_raw.iloc[predict_start].name, factors_raw.iloc[predict_end - 1].name)

print(f"Training dates: {train_dates}")
print(f"Testing dates: {test_dates}")
print(f"Prediction dates: {predict_dates}")

# Split data into training, testing, and prediction sets
train_factors = factors_raw.iloc[train_start:train_end]
train_returns = stock_returns.iloc[train_start:train_end]
test_factors = factors_raw.iloc[test_start:test_end]
test_returns = stock_returns.iloc[test_start:test_end]
predict_factors = factors_raw.iloc[predict_start:predict_end]
predict_returns = stock_returns.iloc[predict_start:predict_end]

# Normalize train and test data
train_factors_mean, train_factors_std = train_factors.mean(), train_factors.std()
train_returns_mean, train_returns_std = train_returns.mean(), train_returns.std()

train_factors = (train_factors - train_factors_mean) / train_factors_std
train_returns = (train_returns - train_returns_mean) / train_returns_std
test_factors = (test_factors - train_factors_mean) / train_factors_std
test_returns = (test_returns - train_returns_mean) / train_returns_std

# Initialize dataset and dataloader
dataset = StockDataset(train_factors, train_returns, sequence_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model initialization
d_model = train_factors.shape[1] 
num_heads = min(4, d_model)  # Ensure num_heads does not exceed d_model and d_model is divisible by num_heads
while d_model % num_heads != 0:
    num_heads -= 1
# print(f"d_model: {d_model} num_heads: {num_heads} sequence_length: {sequence_length} and train factor return dim: {train_factors.shape[1]} {train_returns.shape[1]}")
model = StockPredictAgent(d_model=d_model, num_heads=num_heads, sequence_length=sequence_length).to(device)



In [ ]:

# Optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()


In [ ]:

train_model_with_logging(model, dataloader, optimizer, criterion, num_epochs, device)

In [ ]:
len(test_factors)
sequence_length

In [ ]:
import shap
import torch

test_dataset = StockDataset(test_factors, test_returns, sequence_length)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Wrap the model for SHAP (pass test_returns)
model_wrapper = StockPredictWrapper(model, device, test_returns.to_numpy())

# Use `DeepExplainer`, which is designed for deep learning models
explainer = shap.DeepExplainer(model_wrapper, torch.tensor(test_factors.to_numpy(), dtype=torch.float32).to(device))

model.eval()
with torch.no_grad():
    total_test_loss = 0.0
    for factors, returns, target in test_dataloader:
        factors, returns, target = factors.to(device), returns.to(device), target.to(device)
        predictions = model(factors, returns)
        loss = criterion(predictions, target)
        total_test_loss += loss.item()

    # Compute SHAP values using DeepExplainer
    shap_values = explainer.shap_values(torch.tensor(test_factors.to_numpy(), dtype=torch.float32).to(device))

    avg_test_loss = total_test_loss / len(test_dataloader)
    print(f"Test Loss: {avg_test_loss:.4f}")

# Visualize Feature Importance
shap.summary_plot(shap_values, test_factors)


In [ ]:
test_factors

In [ ]:
import shap
import torch

# Set model to evaluation mode
model.eval()

# Define a PyTorch model wrapper for SHAP
class StockPredictWrapper(torch.nn.Module):
    def __init__(self, model):
        super(StockPredictWrapper, self).__init__()
        self.model = model

    def forward(self, *inputs):
        factors, returns = inputs  # Extract both inputs

        factors = torch.tensor(factors, dtype=torch.float32).to(device)
        returns = torch.tensor(returns, dtype=torch.float32).to(device)

        if len(factors.shape) == 2:
            factors = factors.unsqueeze(1)
        if len(returns.shape) == 2:
            returns = returns.unsqueeze(1)

        with torch.no_grad():
            return self.model(factors, returns).cpu()

# Wrap the model
model_wrapper = StockPredictWrapper(model).to(device)

# Select a subset of training data for background (60 rows)
background_factors = torch.tensor(test_factors.sample(60).values, dtype=torch.float32).to(device)
background_returns = torch.tensor(test_returns.sample(60).values, dtype=torch.float32).to(device)

# Initialize `DeepExplainer` with both inputs
explainer = shap.DeepExplainer(model_wrapper, [background_factors, background_returns])

# Convert test data to tensor
test_factors_tensor = torch.tensor(test_factors.values, dtype=torch.float32).to(device)
test_returns_tensor = torch.tensor(test_returns.values, dtype=torch.float32).to(device)

# Compute SHAP values
shap_values = explainer.shap_values([test_factors_tensor, test_returns_tensor])

# Visualize Feature Importance
shap.summary_plot(shap_values, test_factors)


In [ ]:
import sys
from SVERL_icml_2023.portfolio_DRL.data_function import *
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader
from SVERL_icml_2023.portfolio_DRL.create_model import *
from sklearn.model_selection import TimeSeriesSplit
from SVERL_icml_2023.shapley import Shapley
import shap
sys.path.append("E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023")
# from portfolio_DRL.create_model import StockPredictWrapper


# Example usage
input_dir = "E:\XAI\RL_with_SHAP\pythonProject1\SVERL_icml_2023\portfolio_DRL"  # Use the specified directory
ticker_file = os.path.join(input_dir, "tickers.txt")  # Text file containing tickers, one per line
factor_file = os.path.join(input_dir, "FF_factors.csv")  # CSV file containing factor returns
start_date = "2005-01-01"
end_date = "2014-12-31"
output_csv = os.path.join(input_dir, "stock_prices.csv")

# try:
#     tickers = load_tickers_from_file(ticker_file)
#     price_data = download_stock_prices(tickers, start_date, end_date, output_csv)
#     daily_returns = calculate_daily_returns(price_data, return_type='arithmetic')
#     daily_returns.to_csv(os.path.join(input_dir, "daily_returns.csv"))
#     factor_returns = read_factor_returns(factor_file, daily_returns)
#     factor_returns.to_csv(os.path.join(input_dir, "aligned_factors.csv"))
#     print("Factor returns alignment completed.")
# except Exception as e:
#     print(f"An error occurred: {e}")

factors = pd.read_csv(f"{input_dir}/aligned_factors.csv", index_col=0, parse_dates=True)
returns_raw = pd.read_csv(f"{input_dir}/daily_returns.csv", index_col=0, parse_dates=True)


# Remove NaN values and align dates
factors = factors.dropna()
returns = returns_raw.dropna()

# Align dates between factors and returns
dates = factors.index.intersection(returns.index)
factors_raw = factors.loc[dates]
returns_raw = returns.loc[dates]
factors_raw = factors_raw.iloc[:, :-1]
# print(f"factors_raw = {factors_raw}")

Nday_diff = 10  # Define the future prediction horizon for shift days
# limit to 2 stock ONLY
# returns_raw = returns_raw.iloc[:, :2]

# Remove NaN values and align dates
factors_raw = factors_raw.dropna()
returns_raw = returns_raw.dropna()
# factors_raw = pd.concat([factors_raw, returns_raw], axis=1)
# Align dates between factors_raw and returns_raw
dates = factors_raw.index.intersection(returns_raw.index)
factors_raw = factors_raw.loc[dates]
returns_raw = returns_raw.loc[dates]

# Shift returns to generate target variable (N-day future return)
returns_raw_Y = returns_raw.shift(-Nday_diff).dropna()

# Align all datasets with available dates in returns_raw_Y
dates_final = factors_raw.index.intersection(returns_raw_Y.index)
factors_raw = factors_raw.loc[dates_final]
returns_raw = returns_raw.loc[dates_final]
returns_raw_Y = returns_raw_Y.loc[dates_final]

N = 10
sequence_length = 10
batch_size = 64
num_epochs = 500
learning_rate = 0.001
rolling_window_train = 360  # Reduce for faster adaptation
rolling_window_test = 10    # Reduce test window
prediction_window = 10      # Align prediction period with test window

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Record results
results = []
prediction_records = []
shapley_records = []
shapley_metrics_df = []
all_test_losses = []
all_binary_test_losses = []
results = []
classification_results = []

test_results = []
prediction_results = []
combined_test_result = []
portfolio_records = []
start = 0  # Initialize rolling split index

# state = 'train_data_generate'
state = 'train_RL'
# state = 'final_trade'


if state == 'train_data_generate':
    # Rolling split: Train (M days), Test (N days), Predict (W days)
    total_samples = len(factors_raw)
    while start + rolling_window_train + rolling_window_test + prediction_window <= total_samples:

        train_start = start
        train_end = start + rolling_window_train

        test_start = train_end
        test_end = test_start + rolling_window_test

        predict_start = test_end  # Predictions start right after testing
        predict_end = predict_start + prediction_window

        # Extract the actual date indices
        train_dates = (factors_raw.iloc[train_start].name, factors_raw.iloc[train_end - 1].name)
        test_dates = (factors_raw.iloc[test_start].name, factors_raw.iloc[test_end - 1].name)
        predict_dates = (factors_raw.iloc[predict_start].name, factors_raw.iloc[predict_end - 1].name)

        print(f"\n🔹 Processing Date Window: Train {train_dates}, Test {test_dates}, Predict {predict_dates}")

        for stock in returns_raw.columns:
            print(f"Training for stock: {stock}")
            stock_returns = returns_raw_Y[[stock]]  # Select the stock column
            stock_returns_unshift = returns_raw[[stock]]
            factors_raw_X = pd.concat([factors_raw, stock_returns_unshift], axis=1)
            # Rolling split: Train (M days), Test (N days), Predict (W days)
            # total_samples = len(factors_raw_X)
            # Initialize the rolling split index

            # Print the selected date ranges
            print(f"Training dates: {train_dates}")
            print(f"Testing dates: {test_dates}")
            print(f"Prediction dates: {predict_dates}")

            # # Split data into training, testing, and prediction sets
            train_factors = factors_raw_X.iloc[train_start:train_end]
            train_returns = stock_returns.iloc[train_start:train_end]
            test_factors = factors_raw_X.iloc[test_start:test_end]
            test_returns = stock_returns.iloc[test_start:test_end]
            predict_factors = factors_raw_X.iloc[predict_start:predict_end]
            predict_returns = stock_returns.iloc[predict_start:predict_end]
            #
            # # Normalize train and test data
            train_factors_mean, train_factors_std = train_factors.mean(), train_factors.std()
            train_returns_mean, train_returns_std = train_returns.mean(), train_returns.std()

            # Avoid division by zero
            train_factors_std.replace(0, 1e-8, inplace=True)
            train_returns_std.replace(0, 1e-8, inplace=True)

            train_factors_norm = (train_factors - train_factors_mean) / train_factors_std
            train_returns_norm = (train_returns - train_returns_mean) / train_returns_std
            test_factors_norm = (test_factors - train_factors_mean) / train_factors_std
            test_returns_norm = (test_returns - train_returns_mean) / train_returns_std
            predict_factors_norm = (predict_factors - train_factors_mean) / train_factors_std
            predict_returns_norm = (predict_returns - train_returns_mean) / train_returns_std
            # print(f'train_factors_norm dim: {train_factors_norm.shape}')
            # print(f'train_returns_norm dim: {train_returns_norm.shape}')
            dataset = StockDataset(train_factors_norm, train_returns_norm, sequence_length, N=N)

            dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            # for batch_x, batch_y in dataloader:
            #     print(f"Batch X Shape: {batch_x.shape}")  # Expected: (batch_size, sequence_length, num_features)
            #     print(f"Batch Y Shape: {batch_y.shape}")  # Expected: (batch_size, N, num_features)
            #     break  # Only print for the first batch
            # # Model initialization
            d_model = train_factors.shape[1]
            # print(f'd_model: {d_model}')

            num_heads = min(4,
                            d_model)  # Ensure num_heads does not exceed d_model and d_model is divisible by num_heads
            while d_model % num_heads != 0:
                num_heads -= 1
            # print(f"d_model: {d_model} num_heads: {num_heads} sequence_length: {sequence_length} and train factor return dim: {train_factors.shape[1]} {train_returns.shape[1]}")
            model = StockPredictAgent(d_model=d_model, num_heads=num_heads, sequence_length=sequence_length, N=N).to(
                device)
            #
            # # Optimizer and loss function
            optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
            criterion = nn.SmoothL1Loss(beta=1.0)  # Huber Loss for stability
            #
            # # Training the model
            # print(f"Training window: {train_start} to {train_end}")
            # print(f"Expected FC Layer Input Shape: {sequence_length * d_model}")
            # print(f"Flattened Tensor Shape: {batch_size, sequence_length * d_model}")

            train_model_with_logging(model, dataloader, optimizer, criterion, scheduler, num_epochs, device)

            print(f'Finished training model 1')
            # print(f'train_factors_norm dim: {train_factors_norm.shape}')
            # Convert to PyTorch tensors
            # print(f'test_factors_tensor: {test_factors_tensor.shape}')
            # Run inference on test data
            augmented_train_features = generate_augmented_features(
                train_factors_norm, train_factors_mean, train_factors_std, model, rolling_window_test, device
            )
            augmented_features_tensor = torch.tensor(augmented_train_features, dtype=torch.float32).to(device)

            print(
                f"Final Augmented Tensor Shape: {augmented_train_features.shape}")  # Expected: (rolling_window_test, d_model * 2)

            # Convert test_returns_norm into binary classification
            mean_return_binary = np.mean(train_returns_norm)  # Aggregate 60-day return
            train_returns_threshold = (0 - train_returns_mean) / train_returns_std  # Convert zero into normalized scale
            train_returns_binary = (mean_return_binary > train_returns_threshold).astype(
                int)  # 1 if positive, 0 if negative
            # print(f'train_returns_norm = {train_returns_norm}')
            print(f"Cumulative Return over 60 days: {mean_return_binary}")
            print(f"Binary Target (Up=1, Down=0): {train_returns_binary}")

            # Train the Model Using Augmented Features
            # sequence_length = 20
            # batch_size = 32
            # num_epochs = 25
            # learning_rate = 0.001
            d_model2 = augmented_train_features.shape[1]  # Number of features (6 original + 6 SHAP)
            num_heads = min(4, d_model2)
            while d_model2 % num_heads != 0:
                num_heads -= 1

            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            print(f"Length of augmented_test_features: {augmented_train_features.shape}")
            print(f"Length of sequence: {sequence_length}")

            # sequence_length2 = min(10, len(augmented_test_features))  # Ensure it's smaller than available data
            sequence_length2 = prediction_window
            # Create dataset and dataloader for training
            dataset2 = StockDatasetBinary(augmented_train_features, train_returns_norm, sequence_length2)
            dataloader2 = DataLoader(dataset2, batch_size=batch_size, shuffle=True)

            for batch_x, batch_y in dataloader2:
                print(f"Batch X Shape: {batch_x.shape}")  # Expected: (batch_size, sequence_length, num_features)
                print(f"Batch Y Shape: {batch_y.shape}")  # Expected: (batch_size, N, num_features)
                break  # Only print for the first batch

            # Initialize the binary classification model
            model_binary = StockPredictAgentBinary(d_model=d_model2, num_heads=num_heads,
                                                   sequence_length=sequence_length2).to(device)

            # Define optimizer and loss function
            optimizer = optim.Adam(model_binary.parameters(), lr=learning_rate, weight_decay=1e-5)
            criterion = nn.BCELoss()  # Binary cross-entropy loss

            # Train the model
            train_model(model_binary, dataloader2, optimizer, criterion, num_epochs, device)
            print(f'Finished training model 2')

            test_factors_tensor = torch.tensor(test_factors_norm.values, dtype=torch.float32).to(device).unsqueeze(0)
            predict_factors_tensor = torch.tensor(predict_factors_norm.values, dtype=torch.float32).to(
                device).unsqueeze(0)

            model.eval()
            with torch.no_grad():
                test_predictions_norm = model(test_factors_tensor).cpu().numpy().flatten()
                future_predictions_norm = model(predict_factors_tensor).cpu().numpy().flatten()

                # test_output = model(torch.tensor(test_factors_np, dtype=torch.float32).to(device))
                # print(f"Model output shape: {test_output.shape}")  # Expected: (1, 20) for 20 future returns
            # print(f'test_factors_tensor: {test_factors_tensor.shape}')
            # Revert normalization for test and predicted returns
            test_predictions = test_predictions_norm * train_returns_std.item() + train_returns_mean.item()
            future_predictions = future_predictions_norm * train_returns_std.item() + train_returns_mean.item()

            augmented_test_features = generate_augmented_features_testphase(
                test_factors_norm, train_factors_norm, train_factors_mean, train_factors_std,
                model, prediction_window, device
            )
            # ✅ Convert NumPy array to PyTorch tensor
            augmented_features_tensor_test = torch.tensor(augmented_test_features, dtype=torch.float32).to(device)

            # ✅ Ensure the tensor is 3D by adding a batch dimension at index 0
            augmented_features_tensor_test = augmented_features_tensor_test.unsqueeze(0)  # Shape becomes (1, 20, 12)

            augmented_predict_features = generate_augmented_features_testphase(
                predict_factors_norm, train_factors_norm, train_factors_mean, train_factors_std,
                model, prediction_window, device
            )
            # ✅ Convert NumPy array to PyTorch tensor
            augmented_features_tensor_test = torch.tensor(augmented_test_features, dtype=torch.float32).to(device)
            augmented_features_tensor_predict = torch.tensor(augmented_predict_features, dtype=torch.float32).to(device)

            # ✅ Ensure the tensor is 3D by adding a batch dimension at index 0
            augmented_features_tensor_test = augmented_features_tensor_test.unsqueeze(0)  # Shape becomes (1, 20, 12)
            augmented_features_tensor_predict = augmented_features_tensor_predict.unsqueeze(
                0)  # Shape becomes (1, 20, 12)

            # ✅ Debugging: Print final shape
            print(
                f'✅ Transformed augmented_features_tensor_test Shape: {augmented_features_tensor_test.shape}')  # Expected: (1, 20, 12)

            # Get model predictions
            model_binary.eval()
            with torch.no_grad():
                return_test_prob = model_binary(augmented_features_tensor_test).cpu().numpy().flatten()
                return_predict_prob = model_binary(augmented_features_tensor_predict).cpu().numpy().flatten()

            print(f'fnished 2nd model test')
            # Convert probabilities to binary labels (0 = down, 1 = up)
            # Convert probability outputs to binary labels (0 = down, 1 = up)
            return_test_binary = (return_test_prob > 0.5).astype(int).item()
            return_predict_binary = (return_predict_prob > 0.5).astype(int).item()

            # TO-DO: add test phase stats (loss from both models)
            # TO-DO: add prediction data from both models, find stocks that go ups and downs with their predicted return series

            # future_predictions = future_predictions_norm * train_returns_std.item() + train_returns_mean.item()
            actual_test_returns = test_returns.values.flatten()
            actual_predict_returns = predict_returns.values.flatten()
            actual_test_returns_binary = (np.mean(actual_test_returns) > 0).astype(int)
            actual_predict_returns_binary = (np.mean(actual_predict_returns) > 0).astype(int)

            # print(f'test_predictions dim:{test_predictions.shape} actual_test_returns dim: {actual_test_returns.shape}')

            # Compute loss for test and prediction phases
            test_loss = np.mean((test_predictions - actual_test_returns) ** 2)
            predict_loss = np.mean((future_predictions - actual_predict_returns) ** 2)

            binary_test_loss = np.mean(np.abs(return_test_binary - actual_test_returns_binary))
            binary_predict_loss = np.mean(np.abs(return_predict_binary - actual_predict_returns_binary))

            all_test_losses.append(test_loss)
            print(f'test_loss: {test_loss}')
            all_binary_test_losses.append(binary_test_loss)
            print(f'binary test_loss: {binary_test_loss}')

            # Store test results
            combined_test_result = {
                "Stock": stock,
                "Test Dates": (test_start, test_end),
                "Predicted Returns (Test Set)": test_predictions.tolist(),
                "Actual Returns (Test Set)": actual_test_returns.tolist(),
                "Loss": test_loss
            }

            # Store time series results (date-wise)
            for i in range(len(test_predictions)):
                results.append([
                    stock,
                    "test",
                    test_returns.index[i],  # Store the date
                    test_predictions[i],
                    actual_test_returns[i]
                ])

            for i in range(len(future_predictions)):
                results.append([
                    stock,
                    "predict",
                    predict_returns.index[i],  # Store the date
                    future_predictions[i],
                    actual_predict_returns[i]
                ])

            # calculate 60 days vol for stock i
            stock_historical_returns_test = stock_returns_unshift.iloc[(train_end - 60):train_end]
            stock_historical_returns_predict = stock_returns_unshift.iloc[(test_end - 60):test_end]

            stock_volatility_test = stock_historical_returns_test.std().values[0]
            stock_volatility_predict = stock_historical_returns_predict.std().values[0]

            # Store scalar classification results (with start & end dates)
            classification_results.append([
                stock, "test_binary", test_dates[0], test_dates[1], return_test_binary, return_test_prob,
                actual_test_returns_binary, stock_volatility_test, float(np.prod(test_returns + 1) - 1)
            ])
            classification_results.append([
                stock, "predict_binary", predict_dates[0], predict_dates[1], return_predict_binary, return_predict_prob,
                actual_predict_returns_binary, stock_volatility_predict, float(np.prod(predict_returns + 1) - 1)
            ])

            print(f"Loss for {stock}: Test Loss: {test_loss:.6f}, Predict Loss: {predict_loss:.6f}")
            print(
                f"Binary Model Loss for {stock}: Test Loss: {binary_test_loss:.6f}, Predict Loss: {binary_predict_loss:.6f}")

            # Convert results to DataFrame
            # Add SHAP feature importance metrics to the test results
            # combined_test_result.update(feature_importance_df_pivot.iloc[0].to_dict())

            # Append the combined results to the test_results list
            # test_results.append(combined_test_result)
            #
            # for date, predicted, actual in zip(predict_returns.index, future_predictions, actual_predict_returns):
            #     prediction_results.append({
            #         "Stock": stock,
            #         "Date": date,
            #         "Predicted Return": predicted,
            #         "Actual Return": actual
            #     })
            # print(f"Predictions for {stock} from {predict_start} to {predict_end} complete.")
            # Print average test loss for this stock
            avg_test_loss = np.mean(all_test_losses)
            avg_binary_test_loss = np.mean(all_binary_test_losses)
            print(
                f"Average Test Loss for {stock}: {avg_test_loss:.6f} and Average Binary Test Loss is {avg_binary_test_loss:.6f}")

        # Convert classification_results into a DataFrame
        classification_df = pd.DataFrame(classification_results, columns=[
            "Stock", "Phase", "Start Date", "End Date", "Predicted Binary", "Predicted Probability",
            "Actual Binary", "Volatility", "Next Period Return"
        ])

        # 🔹 Filter for a specific test period (test_dates[1])
        selected_test_date = test_dates[1]  # Replace with the target test end date
        filtered_df = classification_df[(classification_df["Phase"] == "predict_binary") &
                                        (classification_df["End Date"] == selected_test_date)]

        # 🔹 Extract required inputs
        stock_tickers = filtered_df["Stock"].tolist()
        predicted_probs = filtered_df["Predicted Probability"].values.astype(float)
        rolling_volatility = filtered_df["Volatility"].values.astype(float)

        # 🔹 Map stocks to sectors (Assume `sector_dict` contains stock-sector mapping)
        sector_dict = {
            'BA': 'Industrials',
            'AMGN': 'Healthcare',
            'DIS': 'Communication Services',
            'NKE': 'Consumer Discretionary',
            'HON': 'Industrials',
            'MMM': 'Industrials',
            'CAT': 'Industrials',
            'KO': 'Consumer Staples',
            'PG': 'Consumer Staples',
            'AXP': 'Financials',
            'GS': 'Financials',
            'JPM': 'Financials',
            'MCD': 'Consumer Discretionary',
            'HD': 'Consumer Discretionary',
            'AAPL': 'Technology',
            'CRM': 'Technology',
            'CSCO': 'Technology',
            'IBM': 'Technology',
            'MSFT': 'Technology',
            'TRV': 'Financials',
            'UNH': 'Healthcare',
            'CVX': 'Energy',
            'JNJ': 'Healthcare',
            'MRK': 'Healthcare',
            'AMZN': 'Consumer Discretionary',
            'WMT': 'Consumer Staples',
            'INTC': 'Technology',
            'VZ': 'Communication Services'
        }  # Example mapping, replace with real sector data
        sectors = [sector_dict.get(stock, "Unknown") for stock in stock_tickers]  # Default to "Unknown" if missing

        # TO-DO: i can use test period data to fine tune confidence_threshold and sector_limit parameter and then trade on prediction period:
        # 🔹 Call compute_portfolio_weights function
        portfolio_weights = compute_portfolio_weights(
            stock_tickers=stock_tickers,
            predicted_probs=predicted_probs,
            rolling_volatility=rolling_volatility,
            sectors=sectors,
            confidence_threshold=(0.4, 0.6),  # Exclude low-confidence predictions
            sector_limit=0.3,  # Max 30% per sector
            annualize_volatility=False  # Convert daily volatility to annualized if needed
        )

        # 🔹 Display final portfolio allocation
        print("\n🔹 Final Portfolio Weights for Test Date:", selected_test_date)
        print(portfolio_weights)

        # 🔹 Merge all inputs with portfolio weights
        for i in range(len(stock_tickers)):
            portfolio_records.append([
                selected_test_date,  # Store time period
                stock_tickers[i],
                sectors[i],
                predicted_probs[i],
                rolling_volatility[i],
                0.4,  # Confidence threshold lower bound
                0.6,  # Confidence threshold upper bound
                0.3,  # Sector constraint limit
                False,  # Annualized volatility flag
                portfolio_weights.loc[portfolio_weights["Stock"] == stock_tickers[i], "Normalized Weight"].values[0]
                # Extract weight
            ])

        start += prediction_window

    portfolio_results_df = pd.DataFrame(portfolio_records, columns=[
        "Time Period", "Stock", "Sector", "Predicted Probability", "Rolling Volatility",
        "Confidence Threshold (Low)", "Confidence Threshold (High)", "Sector Limit",
        "Annualized Volatility", "Final Portfolio Weight"
    ])

    # Save results to a CSV file
    portfolio_results_df.to_csv("portfolio_results.csv", index=False)
    # Convert to DataFrame
    # Convert to DataFrame
    df_results = pd.DataFrame(results, columns=["Stock", "Phase", "Date", "Predicted Return", "Actual Return"])
    df_classification = pd.DataFrame(classification_results,
                                     columns=["Stock", "Phase", "Start Date", "End Date", "Predicted Binary",
                                              "Predict Probability", "Actual Binary", "Stock Volatility",
                                              "Next Period Return"])

    # Save results to CSV
    df_results.to_csv("stock_predictions_results.csv", index=False)
    df_classification.to_csv("stock_predictions_classification.csv", index=False)

    print("Results saved successfully!")

elif state == 'train_RL':

    # Load sample stock data (Replace with actual data loading)
    stock_data = pd.read_csv("sample_stock_data.csv")  # Assume CSV file with test data

     # Train the RL agent
    trained_model = train_rl_agent(stock_data, max_periods=213, num_episodes=5000, N=100)
    
    # Get averaged optimal parameters
    M = 100  # Number of samples to generate
    optimal_params = get_average_optimal_parameters(trained_model, stock_data, num_periods=10, M=M)
    
    print("Averaged Optimized Confidence Threshold:", optimal_params[0])
    print("Averaged Optimized Sector Limit:", optimal_params[1])
    
    # Save averaged optimal parameters to CSV
    optimal_params_df = pd.DataFrame({
        'Confidence Threshold Low': [optimal_params[0][0]],
        'Confidence Threshold High': [optimal_params[0][1]],
        'Sector Limit': [optimal_params[1]]
    })
    optimal_params_df.to_csv("optimal_parameters.csv", index=False)
    
    print("Optimal parameters saved to optimal_parameters.csv")




In [ ]:
import sys
from SVERL_icml_2023.portfolio_DRL.data_function import *
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader
from SVERL_icml_2023.portfolio_DRL.create_model import *
from sklearn.model_selection import TimeSeriesSplit
from SVERL_icml_2023.shapley import Shapley
import shap
sys.path.append("E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023")
# from portfolio_DRL.create_model import StockPredictWrapper


# Example usage
input_dir = "E:\XAI\RL_with_SHAP\pythonProject1\SVERL_icml_2023\portfolio_DRL"  # Use the specified directory
ticker_file = os.path.join(input_dir, "tickers.txt")  # Text file containing tickers, one per line
factor_file = os.path.join(input_dir, "FF_factors.csv")  # CSV file containing factor returns
start_date = "2015-01-01"
end_date = "2024-12-31"
output_csv = os.path.join(input_dir, "stock_prices.csv")

try:
    tickers = load_tickers_from_file(ticker_file)
    price_data = download_stock_prices(tickers, start_date, end_date, output_csv)
    daily_returns = calculate_daily_returns(price_data, return_type='arithmetic')
    daily_returns.to_csv(os.path.join(input_dir, "daily_returns.csv"))
    factor_returns = read_factor_returns(factor_file, daily_returns)
    factor_returns.to_csv(os.path.join(input_dir, "aligned_factors.csv"))
    print("Factor returns alignment completed.")
except Exception as e:
    print(f"An error occurred: {e}")

factors = pd.read_csv(f"{input_dir}/aligned_factors.csv", index_col=0, parse_dates=True)
returns_raw = pd.read_csv(f"{input_dir}/daily_returns.csv", index_col=0, parse_dates=True)


# Remove NaN values and align dates
factors = factors.dropna()
returns = returns_raw.dropna()

# Align dates between factors and returns
dates = factors.index.intersection(returns.index)
factors_raw = factors.loc[dates]
returns_raw = returns.loc[dates]
factors_raw = factors_raw.iloc[:, :-1]

In [ ]:
factors_raw

In [1]:
import pandas as pd
import numpy as np

# Load the data
weights_file = "port_wt_test.csv"  # Optimal security weights
returns_file = "port_ret_test.csv"  # Daily stock returns

# Read the files
weights_df = pd.read_csv(weights_file)
returns_df = pd.read_csv(returns_file)

# Convert Date columns to datetime format for proper alignment
weights_df["Date"] = pd.to_datetime(weights_df["Date"], format="%m/%d/%Y")
returns_df["Date"] = pd.to_datetime(returns_df["Date"], format="%m/%d/%Y")

# Set Date as index for easier merging
weights_df.set_index("Date", inplace=True)
returns_df.set_index("Date", inplace=True)

# Sort data to ensure chronological order
weights_df.sort_index(inplace=True)
returns_df.sort_index(inplace=True)

# Initialize drifted weights dataframe with the same structure as returns_df
aligned_weights_df = weights_df.reindex(returns_df.index, method='ffill')
drifted_weights_df = aligned_weights_df.copy()

# Iterate through each day to apply drifted weight calculation
for i in range(1, len(drifted_weights_df)):
    # If the day is a rebalancing day, keep the given weights
    if drifted_weights_df.index[i] in weights_df.index:
        continue

    # Otherwise, update weights based on previous day's weights and returns
    drifted_weights_df.iloc[i] = drifted_weights_df.iloc[i - 1] * (1 + returns_df.iloc[i - 1])

    # Normalize weights to ensure they sum to 100%
    drifted_weights_df.iloc[i] /= drifted_weights_df.iloc[i].sum()

# Compute portfolio returns using drifted weights
portfolio_returns_drifted = (returns_df * drifted_weights_df).sum(axis=1)

# Save portfolio returns to CSV
portfolio_returns_drifted.to_csv("portfolio_returns_drifted.csv")
# Display first few rows
print(portfolio_returns_drifted.head())


Date
2009-07-16    0.007590
2009-07-17    0.000810
2009-07-20    0.008193
2009-07-21   -0.001259
2009-07-22    0.003083
dtype: float64


In [ ]:
drifted_weights_df.to_csv("portfolio_returns_driftedweights.csv")


In [5]:
import sys
from SVERL_icml_2023.portfolio_DRL.data_function import *
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader
from SVERL_icml_2023.portfolio_DRL.create_model import *
from sklearn.model_selection import TimeSeriesSplit
from SVERL_icml_2023.shapley import Shapley
import shap
sys.path.append("E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023")
# from portfolio_DRL.create_model import StockPredictWrapper
input_dir = "E:\XAI\RL_with_SHAP\pythonProject1\SVERL_icml_2023\portfolio_DRL"  # Use the specified directory

start_date = "2009-07-16"
end_date = "2024-12-31"
output_csv = os.path.join(input_dir, "stock_prices_DIA.csv")

ticker_file = os.path.join(input_dir, "tickers_DIA.txt")  # Text file containing tickers, one per line
print(ticker_file)
tickers = load_tickers_from_file(ticker_file)
print(tickers)
price_data = download_stock_prices(tickers, start_date, end_date, output_csv)

print("Factor returns alignment completed.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

E:\XAI\RL_with_SHAP\pythonProject1\SVERL_icml_2023\portfolio_DRL\tickers_DIA.txt
['AAPL', 'DIA', 'IYR']



[*********************100%***********************]  1 of 1 completed

Data saved to E:\XAI\RL_with_SHAP\pythonProject1\SVERL_icml_2023\portfolio_DRL\stock_prices_DIA.csv
Factor returns alignment completed.


In [ ]:
import pandas as pd
import numpy as np
def compute_portfolio_weights(stock_tickers, predicted_probs, rolling_volatility, sectors,
                              confidence_threshold=(0.4, 0.6), short_ratio=0.25,
                              annualize_volatility=False):
    """
    Compute optimal portfolio weights for long-short portfolio ensuring total weight sums to 100%
    while maintaining the short ratio.

    Parameters:
    - stock_tickers (list): List of stock tickers.
    - predicted_probs (np.array): Predicted probability of stock moving up.
    - rolling_volatility (np.array): Rolling volatility (daily or annualized).
    - sectors (list): List of sector classifications for each stock.
    - confidence_threshold (tuple): Threshold (low, high) to filter out uncertain predictions.
    - short_ratio (float): Proportion of short exposure (between 0 and 50%).
    - annualize_volatility (bool): Whether to annualize daily volatility using sqrt(252).

    Returns:
    - pd.DataFrame: Portfolio allocation with normalized long-short weights.
    """
    # Convert probability into raw weights for long-short portfolio
    raw_weights = 2 * (predicted_probs - 0.5)

    # Apply confidence threshold: Only include confident stocks
    confidence_mask = (predicted_probs >= confidence_threshold[1]) | (predicted_probs <= confidence_threshold[0])
    # print(f'confidence_mask = {confidence_mask}')
    filtered_weights = raw_weights * confidence_mask  # Zero out low-confidence predictions
    # print(f'filtered_weights = {filtered_weights}')

    # Check if no stock qualifies
    if np.all(filtered_weights == 0):
        raise ValueError("No stocks passed the confidence threshold; cannot generate portfolio.")

    # Annualize volatility if required
    if annualize_volatility:
        rolling_volatility = rolling_volatility * np.sqrt(252)

    # Apply Volatility Adjustment: Wi,Tadjusted = Wi,T / σi
    adjusted_weights = filtered_weights / rolling_volatility  # Scale by inverse volatility

    # Separate long and short portfolios based on short ratio
    long_weights = np.where(adjusted_weights > 0, adjusted_weights, 0)
    short_weights = np.where(adjusted_weights < 0, adjusted_weights, 0)

    # Normalize long and short weights so they sum to the target allocations
    long_sum = np.sum(long_weights)
    short_sum = np.abs(np.sum(short_weights))

    if long_sum > 0:
        long_weights *= (1 + short_ratio) / long_sum  # Scale long portfolio to (1 + short_ratio)
    if short_sum > 0:
        short_weights *= short_ratio / short_sum  # Scale short portfolio to -short_ratio

    # Compute final portfolio weights (long + short should sum to 100%)
    final_weights = long_weights + short_weights
    # final_weights /= np.sum(np.abs(final_weights))  # Normalize to ensure exact sum of 100%

    # Create portfolio DataFrame
    portfolio_df = pd.DataFrame({
        "Stock": stock_tickers,
        "Sector": sectors,
        "Probability": predicted_probs,
        "Raw Weight": raw_weights,
        "Volatility": rolling_volatility,
        "Adjusted Weight": adjusted_weights,
        "Final Weight": final_weights
    })
    # print(f'portfolio_wt = {np.sum(portfolio_df["Final Weight"])}')
    # print(f'portfolio_short_ratio = {np.abs(np.sum(portfolio_df[portfolio_df["Final Weight"] < 0]["Final Weight"])) }')
    # print(f'total_portfolio_weight = {np.sum(portfolio_df["Final Weight"]) }')

    # Validate constraints
    total_portfolio_weight = np.sum(portfolio_df["Final Weight"])
    assert np.isclose(total_portfolio_weight, 1.0,
                      atol=1e-6), f"Error: Total Portfolio Weight is {total_portfolio_weight}, should be 1.0."

    actual_short_ratio = np.abs(np.sum(portfolio_df[portfolio_df["Final Weight"] < 0]["Final Weight"])) / total_portfolio_weight
    assert np.isclose(actual_short_ratio, short_ratio,
                      atol=1e-6), f"Error: Actual Short Ratio is {actual_short_ratio}, expected {short_ratio}."
    # print(f'final weight : {portfolio_df[["Final Weight"]]}')
    return portfolio_df[["Stock", "Sector", "Probability", "Final Weight"]]


# Generate an example where no stock passes the confidence threshold filter
# Generate synthetic stock tickers
stock_tickers = [f"Stock_{i}" for i in range(10)]

# Generate random predicted probabilities (between 0 and 1)
predicted_probs = np.random.uniform(0.45, 0.55, 10)  # Values centered within 0.45-0.55 range

# Generate random rolling volatility (between 0.1 and 0.5)
rolling_volatility = np.random.uniform(0.1, 0.5, 10)

# Assign random sectors from a predefined list
sectors = np.random.choice(["Tech", "Finance", "Healthcare", "Energy"], size=10)

# Use a very wide confidence threshold to exclude all stocks
wide_confidence_threshold = (0.5, 0.5)  # No stock will qualify as long or short

# Run the function
try:
    portfolio_df = compute_portfolio_weights(
        stock_tickers=stock_tickers,
        predicted_probs=predicted_probs,
        rolling_volatility=rolling_volatility,
        sectors=sectors,
        confidence_threshold=wide_confidence_threshold,  # No stock qualifies
        short_ratio=0,
        annualize_volatility=False
    )
    print("Portfolio successfully generated.")
    print(portfolio_df)
except ValueError as e:
    print(f"Portfolio generation failed: {e}")

In [ ]:
import sys
from SVERL_icml_2023.portfolio_DRL.data_function import *
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader
from SVERL_icml_2023.portfolio_DRL.create_model import *
from sklearn.model_selection import TimeSeriesSplit
from SVERL_icml_2023.shapley import Shapley
import shap
sys.path.append("E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023")
# from portfolio_DRL.create_model import StockPredictWrapper

# Example usage
input_dir = "E:\XAI\RL_with_SHAP\pythonProject1\SVERL_icml_2023\portfolio_DRL"  # Use the specified directory
ticker_file = os.path.join(input_dir, "tickers.txt")  # Text file containing tickers, one per line
factor_file = os.path.join(input_dir, "FF_factors.csv")  # CSV file containing factor returns
start_date = "2015-01-01"
end_date = "2024-12-31"
output_csv = os.path.join(input_dir, "stock_prices.csv")

try:
    tickers = load_tickers_from_file(ticker_file)
    price_data = download_stock_prices(tickers, start_date, end_date, output_csv)
    daily_returns = calculate_daily_returns(price_data, return_type='arithmetic')
    daily_returns.to_csv(os.path.join(input_dir, "daily_returns.csv"))
    factor_returns = read_factor_returns(factor_file, daily_returns)
    factor_returns.to_csv(os.path.join(input_dir, "aligned_factors.csv"))
    print("Factor returns alignment completed.")
except Exception as e:
    print(f"An error occurred: {e}")

factors = pd.read_csv(f"{input_dir}/aligned_factors.csv", index_col=0, parse_dates=True)
returns_raw = pd.read_csv(f"{input_dir}/daily_returns.csv", index_col=0, parse_dates=True)


# Remove NaN values and align dates
factors = factors.dropna()
returns = returns_raw.dropna()

# Align dates between factors and returns
dates = factors.index.intersection(returns.index)
factors_raw = factors.loc[dates]
returns_raw = returns.loc[dates]
factors_raw = factors_raw.iloc[:, :-1]
# print(f"factors_raw = {factors_raw}")

Nday_diff = 10  # Define the future prediction horizon for shift days
# limit to 2 stock ONLY
# returns_raw = returns_raw.iloc[:, :2]

# Remove NaN values and align dates
factors_raw = factors_raw.dropna()
returns_raw = returns_raw.dropna()
# factors_raw = pd.concat([factors_raw, returns_raw], axis=1)
# Align dates between factors_raw and returns_raw
dates = factors_raw.index.intersection(returns_raw.index)
factors_raw = factors_raw.loc[dates]
returns_raw = returns_raw.loc[dates]

# Shift returns to generate target variable (N-day future return)
returns_raw_Y = returns_raw.shift(-Nday_diff).dropna()

# Align all datasets with available dates in returns_raw_Y
dates_final = factors_raw.index.intersection(returns_raw_Y.index)
factors_raw = factors_raw.loc[dates_final]
returns_raw = returns_raw.loc[dates_final]
returns_raw_Y = returns_raw_Y.loc[dates_final]

N = 10
sequence_length = 10
batch_size = 64
num_epochs = 500
learning_rate = 0.001
rolling_window_train = 360  # Reduce for faster adaptation
rolling_window_test = 10    # Reduce test window
prediction_window = 10      # Align prediction period with test window

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Record results
results = []
prediction_records = []
shapley_records = []
shapley_metrics_df = []
all_test_losses = []
all_binary_test_losses = []
results = []
classification_results = []

test_results = []
prediction_results = []
combined_test_result = []
portfolio_records = []
start = 0  # Initialize rolling split index

# state = 'train_data_generate'
state = 'train_data_generate'
# state = 'train_RL'


if state == 'train_data_generate':
    # Rolling split: Train (M days), Test (N days), Predict (W days)
    total_samples = len(factors_raw)
    while start + rolling_window_train + rolling_window_test + prediction_window <= total_samples:

        train_start = start
        train_end = start + rolling_window_train

        test_start = train_end
        test_end = test_start + rolling_window_test

        predict_start = test_end  # Predictions start right after testing
        predict_end = predict_start + prediction_window

        # Extract the actual date indices
        train_dates = (factors_raw.iloc[train_start].name, factors_raw.iloc[train_end - 1].name)
        test_dates = (factors_raw.iloc[test_start].name, factors_raw.iloc[test_end - 1].name)
        predict_dates = (factors_raw.iloc[predict_start].name, factors_raw.iloc[predict_end - 1].name)

        print(f"\n🔹 Processing Date Window: Train {train_dates}, Test {test_dates}, Predict {predict_dates}")

        for stock in returns_raw.columns:
            print(f"Training for stock: {stock}")
            stock_returns = returns_raw_Y[[stock]]  # Select the stock column
            stock_returns_unshift = returns_raw[[stock]]
            factors_raw_X = pd.concat([factors_raw, stock_returns_unshift], axis=1)
            # Rolling split: Train (M days), Test (N days), Predict (W days)
            # total_samples = len(factors_raw_X)
            # Initialize the rolling split index

            # Print the selected date ranges
            print(f"Training dates: {train_dates}")
            print(f"Testing dates: {test_dates}")
            print(f"Prediction dates: {predict_dates}")

            # # Split data into training, testing, and prediction sets
            train_factors = factors_raw_X.iloc[train_start:train_end]
            train_returns = stock_returns.iloc[train_start:train_end]
            test_factors = factors_raw_X.iloc[test_start:test_end]
            test_returns = stock_returns.iloc[test_start:test_end]
            predict_factors = factors_raw_X.iloc[predict_start:predict_end]
            predict_returns = stock_returns.iloc[predict_start:predict_end]
            #
            # # Normalize train and test data
            train_factors_mean, train_factors_std = train_factors.mean(), train_factors.std()
            train_returns_mean, train_returns_std = train_returns.mean(), train_returns.std()

            # Avoid division by zero
            train_factors_std.replace(0, 1e-8, inplace=True)
            train_returns_std.replace(0, 1e-8, inplace=True)

            train_factors_norm = (train_factors - train_factors_mean) / train_factors_std
            train_returns_norm = (train_returns - train_returns_mean) / train_returns_std
            test_factors_norm = (test_factors - train_factors_mean) / train_factors_std
            test_returns_norm = (test_returns - train_returns_mean) / train_returns_std
            predict_factors_norm = (predict_factors - train_factors_mean) / train_factors_std
            predict_returns_norm = (predict_returns - train_returns_mean) / train_returns_std
            # print(f'train_factors_norm dim: {train_factors_norm.shape}')
            # print(f'train_returns_norm dim: {train_returns_norm.shape}')
            dataset = StockDataset(train_factors_norm, train_returns_norm, sequence_length, N=N)

            dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            # for batch_x, batch_y in dataloader:
            #     print(f"Batch X Shape: {batch_x.shape}")  # Expected: (batch_size, sequence_length, num_features)
            #     print(f"Batch Y Shape: {batch_y.shape}")  # Expected: (batch_size, N, num_features)
            #     break  # Only print for the first batch
            # # Model initialization
            d_model = train_factors.shape[1]
            # print(f'd_model: {d_model}')

            num_heads = min(4,
                            d_model)  # Ensure num_heads does not exceed d_model and d_model is divisible by num_heads
            while d_model % num_heads != 0:
                num_heads -= 1
            # print(f"d_model: {d_model} num_heads: {num_heads} sequence_length: {sequence_length} and train factor return dim: {train_factors.shape[1]} {train_returns.shape[1]}")
            model = StockPredictAgent(d_model=d_model, num_heads=num_heads, sequence_length=sequence_length, N=N).to(
                device)
            #
            # # Optimizer and loss function
            optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
            criterion = nn.SmoothL1Loss(beta=1.0)  # Huber Loss for stability
            #
            # # Training the model
            # print(f"Training window: {train_start} to {train_end}")
            # print(f"Expected FC Layer Input Shape: {sequence_length * d_model}")
            # print(f"Flattened Tensor Shape: {batch_size, sequence_length * d_model}")

            train_model_with_logging(model, dataloader, optimizer, criterion, scheduler, num_epochs, device)

            print(f'Finished training model 1')
            # print(f'train_factors_norm dim: {train_factors_norm.shape}')
            # Convert to PyTorch tensors
            # print(f'test_factors_tensor: {test_factors_tensor.shape}')
            # Run inference on test data
            augmented_train_features = generate_augmented_features(
                train_factors_norm, train_factors_mean, train_factors_std, model, rolling_window_test, device
            )
            augmented_features_tensor = torch.tensor(augmented_train_features, dtype=torch.float32).to(device)

            print(
                f"Final Augmented Tensor Shape: {augmented_train_features.shape}")  # Expected: (rolling_window_test, d_model * 2)

            # Convert test_returns_norm into binary classification
            mean_return_binary = np.mean(train_returns_norm)  # Aggregate 60-day return
            train_returns_threshold = (0 - train_returns_mean) / train_returns_std  # Convert zero into normalized scale
            train_returns_binary = (mean_return_binary > train_returns_threshold).astype(
                int)  # 1 if positive, 0 if negative
            # print(f'train_returns_norm = {train_returns_norm}')
            print(f"Cumulative Return over 60 days: {mean_return_binary}")
            print(f"Binary Target (Up=1, Down=0): {train_returns_binary}")

            # Train the Model Using Augmented Features
            # sequence_length = 20
            # batch_size = 32
            # num_epochs = 25
            # learning_rate = 0.001
            d_model2 = augmented_train_features.shape[1]  # Number of features (6 original + 6 SHAP)
            num_heads = min(4, d_model2)
            while d_model2 % num_heads != 0:
                num_heads -= 1

            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            print(f"Length of augmented_test_features: {augmented_train_features.shape}")
            print(f"Length of sequence: {sequence_length}")

            # sequence_length2 = min(10, len(augmented_test_features))  # Ensure it's smaller than available data
            sequence_length2 = prediction_window
            # Create dataset and dataloader for training
            dataset2 = StockDatasetBinary(augmented_train_features, train_returns_norm, sequence_length2)
            dataloader2 = DataLoader(dataset2, batch_size=batch_size, shuffle=True)

            for batch_x, batch_y in dataloader2:
                print(f"Batch X Shape: {batch_x.shape}")  # Expected: (batch_size, sequence_length, num_features)
                print(f"Batch Y Shape: {batch_y.shape}")  # Expected: (batch_size, N, num_features)
                break  # Only print for the first batch

            # Initialize the binary classification model
            model_binary = StockPredictAgentBinary(d_model=d_model2, num_heads=num_heads,
                                                   sequence_length=sequence_length2).to(device)

            # Define optimizer and loss function
            optimizer = optim.Adam(model_binary.parameters(), lr=learning_rate, weight_decay=1e-5)
            criterion = nn.BCELoss()  # Binary cross-entropy loss

            # Train the model
            train_model(model_binary, dataloader2, optimizer, criterion, num_epochs, device)
            print(f'Finished training model 2')

            test_factors_tensor = torch.tensor(test_factors_norm.values, dtype=torch.float32).to(device).unsqueeze(0)
            predict_factors_tensor = torch.tensor(predict_factors_norm.values, dtype=torch.float32).to(
                device).unsqueeze(0)

            model.eval()
            with torch.no_grad():
                test_predictions_norm = model(test_factors_tensor).cpu().numpy().flatten()
                future_predictions_norm = model(predict_factors_tensor).cpu().numpy().flatten()

                # test_output = model(torch.tensor(test_factors_np, dtype=torch.float32).to(device))
                # print(f"Model output shape: {test_output.shape}")  # Expected: (1, 20) for 20 future returns
            # print(f'test_factors_tensor: {test_factors_tensor.shape}')
            # Revert normalization for test and predicted returns
            test_predictions = test_predictions_norm * train_returns_std.item() + train_returns_mean.item()
            future_predictions = future_predictions_norm * train_returns_std.item() + train_returns_mean.item()

            augmented_test_features = generate_augmented_features_testphase(
                test_factors_norm, train_factors_norm, train_factors_mean, train_factors_std,
                model, prediction_window, device
            )
            # ✅ Convert NumPy array to PyTorch tensor
            augmented_features_tensor_test = torch.tensor(augmented_test_features, dtype=torch.float32).to(device)

            # ✅ Ensure the tensor is 3D by adding a batch dimension at index 0
            augmented_features_tensor_test = augmented_features_tensor_test.unsqueeze(0)  # Shape becomes (1, 20, 12)

            augmented_predict_features = generate_augmented_features_testphase(
                predict_factors_norm, train_factors_norm, train_factors_mean, train_factors_std,
                model, prediction_window, device
            )
            # ✅ Convert NumPy array to PyTorch tensor
            augmented_features_tensor_test = torch.tensor(augmented_test_features, dtype=torch.float32).to(device)
            augmented_features_tensor_predict = torch.tensor(augmented_predict_features, dtype=torch.float32).to(device)

            # ✅ Ensure the tensor is 3D by adding a batch dimension at index 0
            augmented_features_tensor_test = augmented_features_tensor_test.unsqueeze(0)  # Shape becomes (1, 20, 12)
            augmented_features_tensor_predict = augmented_features_tensor_predict.unsqueeze(
                0)  # Shape becomes (1, 20, 12)

            # ✅ Debugging: Print final shape
            print(
                f'✅ Transformed augmented_features_tensor_test Shape: {augmented_features_tensor_test.shape}')  # Expected: (1, 20, 12)

            # Get model predictions
            model_binary.eval()
            with torch.no_grad():
                return_test_prob = model_binary(augmented_features_tensor_test).cpu().numpy().flatten()
                return_predict_prob = model_binary(augmented_features_tensor_predict).cpu().numpy().flatten()

            print(f'fnished 2nd model test')
            # Convert probabilities to binary labels (0 = down, 1 = up)
            # Convert probability outputs to binary labels (0 = down, 1 = up)
            return_test_binary = (return_test_prob > 0.5).astype(int).item()
            return_predict_binary = (return_predict_prob > 0.5).astype(int).item()

            # TO-DO: add test phase stats (loss from both models)
            # TO-DO: add prediction data from both models, find stocks that go ups and downs with their predicted return series

            # future_predictions = future_predictions_norm * train_returns_std.item() + train_returns_mean.item()
            actual_test_returns = test_returns.values.flatten()
            actual_predict_returns = predict_returns.values.flatten()
            actual_test_returns_binary = (np.mean(actual_test_returns) > 0).astype(int)
            actual_predict_returns_binary = (np.mean(actual_predict_returns) > 0).astype(int)

            # print(f'test_predictions dim:{test_predictions.shape} actual_test_returns dim: {actual_test_returns.shape}')

            # Compute loss for test and prediction phases
            test_loss = np.mean((test_predictions - actual_test_returns) ** 2)
            predict_loss = np.mean((future_predictions - actual_predict_returns) ** 2)

            binary_test_loss = np.mean(np.abs(return_test_binary - actual_test_returns_binary))
            binary_predict_loss = np.mean(np.abs(return_predict_binary - actual_predict_returns_binary))

            all_test_losses.append(test_loss)
            print(f'test_loss: {test_loss}')
            all_binary_test_losses.append(binary_test_loss)
            print(f'binary test_loss: {binary_test_loss}')

            # Store test results
            combined_test_result = {
                "Stock": stock,
                "Test Dates": (test_start, test_end),
                "Predicted Returns (Test Set)": test_predictions.tolist(),
                "Actual Returns (Test Set)": actual_test_returns.tolist(),
                "Loss": test_loss
            }

            # Store time series results (date-wise)
            for i in range(len(test_predictions)):
                results.append([
                    stock,
                    "test",
                    test_returns.index[i],  # Store the date
                    test_predictions[i],
                    actual_test_returns[i]
                ])

            for i in range(len(future_predictions)):
                results.append([
                    stock,
                    "predict",
                    predict_returns.index[i],  # Store the date
                    future_predictions[i],
                    actual_predict_returns[i]
                ])

            # calculate 60 days vol for stock i
            stock_historical_returns_test = stock_returns_unshift.iloc[(train_end - 60):train_end]
            stock_historical_returns_predict = stock_returns_unshift.iloc[(test_end - 60):test_end]

            stock_volatility_test = stock_historical_returns_test.std().values[0]
            stock_volatility_predict = stock_historical_returns_predict.std().values[0]

            # Store scalar classification results (with start & end dates)
            classification_results.append([
                stock, "test_binary", test_dates[0], test_dates[1], return_test_binary, return_test_prob,
                actual_test_returns_binary, stock_volatility_test, float(np.prod(test_returns + 1) - 1)

            ])
            classification_results.append([
                stock, "predict_binary", predict_dates[0], predict_dates[1], return_predict_binary, return_predict_prob,
                actual_predict_returns_binary, stock_volatility_predict, float(np.prod(predict_returns + 1) - 1)
            ])
            print(f'return_predict_prob = {return_predict_prob}')
            print(f"Loss for {stock}: Test Loss: {test_loss:.6f}, Predict Loss: {predict_loss:.6f}")
            print(
                f"Binary Model Loss for {stock}: Test Loss: {binary_test_loss:.6f}, Predict Loss: {binary_predict_loss:.6f}")

            # Convert results to DataFrame
            # Add SHAP feature importance metrics to the test results
            # combined_test_result.update(feature_importance_df_pivot.iloc[0].to_dict())

            # Append the combined results to the test_results list
            # test_results.append(combined_test_result)
            #
            # for date, predicted, actual in zip(predict_returns.index, future_predictions, actual_predict_returns):
            #     prediction_results.append({
            #         "Stock": stock,
            #         "Date": date,
            #         "Predicted Return": predicted,
            #         "Actual Return": actual
            #     })
            # print(f"Predictions for {stock} from {predict_start} to {predict_end} complete.")
            # Print average test loss for this stock
            avg_test_loss = np.mean(all_test_losses)
            avg_binary_test_loss = np.mean(all_binary_test_losses)
            print(
                f"Average Test Loss for {stock}: {avg_test_loss:.6f} and Average Binary Test Loss is {avg_binary_test_loss:.6f}")

        # Convert classification_results into a DataFrame
        classification_df = pd.DataFrame(classification_results, columns=[
            "Stock", "Phase", "Start Date", "End Date", "Predicted Binary", "Predicted Probability",
            "Actual Binary", "Volatility", "Next Period Return"
        ])

        # 🔹 Filter for a specific test period (test_dates[1])
        selected_test_date = predict_dates[1]  # Replace with the target test end date
        filtered_df = classification_df[(classification_df["Phase"] == "predict_binary") &
                                        (classification_df["End Date"] == selected_test_date)]

        # 🔹 Extract required inputs
        stock_tickers = filtered_df["Stock"].tolist()
        predicted_probs = filtered_df["Predicted Probability"].values.astype(float)
        rolling_volatility = filtered_df["Volatility"].values.astype(float)
        print(f'predicted_probs={predicted_probs}')
        # 🔹 Map stocks to sectors (Assume `sector_dict` contains stock-sector mapping)
        sector_dict = {
            'BA': 'Industrials',
            'AMGN': 'Healthcare',
            'DIS': 'Communication Services',
            'NKE': 'Consumer Discretionary',
            'HON': 'Industrials',
            'MMM': 'Industrials',
            'CAT': 'Industrials',
            'KO': 'Consumer Staples',
            'PG': 'Consumer Staples',
            'AXP': 'Financials',
            'GS': 'Financials',
            'JPM': 'Financials',
            'MCD': 'Consumer Discretionary',
            'HD': 'Consumer Discretionary',
            'AAPL': 'Technology',
            'CRM': 'Technology',
            'CSCO': 'Technology',
            'IBM': 'Technology',
            'MSFT': 'Technology',
            'TRV': 'Financials',
            'UNH': 'Healthcare',
            'CVX': 'Energy',
            'JNJ': 'Healthcare',
            'MRK': 'Healthcare',
            'AMZN': 'Consumer Discretionary',
            'WMT': 'Consumer Staples',
            'INTC': 'Technology',
            'VZ': 'Communication Services'
        }  # Example mapping, replace with real sector data
        sectors = [sector_dict.get(stock, "Unknown") for stock in stock_tickers]  # Default to "Unknown" if missing

        # TO-DO: i can use test period data to fine tune confidence_threshold and sector_limit parameter and then trade on prediction period:
        # 🔹 Call compute_portfolio_weights function
        portfolio_weights = compute_portfolio_weights(
            stock_tickers=stock_tickers,
            predicted_probs=predicted_probs,
            rolling_volatility=rolling_volatility,
            sectors=sectors,
            confidence_threshold=(0.353392545,	0.662810753),  # Exclude low-confidence predictions
            short_ratio=0.262342442,  # Max 30% per sector
            annualize_volatility=False  # Convert daily volatility to annualized if needed
        )
        bmk_weights_abs_wt = compute_portfolio_weights_BMK(
            stock_tickers=stock_tickers,
            predicted_probs=predicted_probs,
            rolling_volatility=rolling_volatility,
            sectors=sectors,
            confidence_threshold=(0.0, 0.0),  # Exclude low-confidence predictions
            short_ratio=0,  # Max 30% per sector
            annualize_volatility=False,
            raw_weight_version =1
        )
        bmk_weights_raw_pred = compute_portfolio_weights_BMK(
            stock_tickers=stock_tickers,
            predicted_probs=predicted_probs,
            rolling_volatility=rolling_volatility,
            sectors=sectors,
            confidence_threshold=(0.0, 0.0),  # Exclude low-confidence predictions
            short_ratio=0,  # Max 30% per sector
            annualize_volatility=False,
            raw_weight_version=2
        )
        # 🔹 Display final portfolio allocation
        print("\n🔹 Final Portfolio Weights for Test Date:", selected_test_date)
        print(portfolio_weights)

        # 🔹 Merge all inputs with portfolio weights
        for i in range(len(stock_tickers)):
            portfolio_records.append([
                selected_test_date,  # Store time period
                stock_tickers[i],
                sectors[i],
                predicted_probs[i],
                rolling_volatility[i],
                0.353392545,  # Confidence threshold lower bound
                0.662810753,  # Confidence threshold upper bound
                0.262342442,  # Sector constraint limit
                False,  # Annualized volatility flag
                portfolio_weights.loc[portfolio_weights["Stock"] == stock_tickers[i], "Final Weight"].values[0],
                bmk_weights_abs_wt.loc[bmk_weights_abs_wt["Stock"] == stock_tickers[i], "Final Weight"].values[0],
                bmk_weights_raw_pred.loc[bmk_weights_raw_pred["Stock"] == stock_tickers[i], "Final Weight"].values[0]

                # Extract weight
            ])

        start += prediction_window

    portfolio_results_df = pd.DataFrame(portfolio_records, columns=[
        "Time Period", "Stock", "Sector", "Predicted Probability", "Rolling Volatility",
        "Confidence Threshold (Low)", "Confidence Threshold (High)", "Short Ratio",
        "Annualized Volatility", "Final Portfolio Weight", "BM Weight Absolute Weight Scale", "BM Weight Raw Preditor Scale"
    ])

    # Save results to a CSV file
    portfolio_results_df.to_csv("portfolio_results.csv", index=False)
    # Convert to DataFrame
    # Convert to DataFrame
    df_results = pd.DataFrame(results, columns=["Stock", "Phase", "Date", "Predicted Return", "Actual Return"])
    df_classification = pd.DataFrame(classification_results,
                                     columns=["Stock", "Phase", "Start Date", "End Date", "Predicted Binary",
                                              "Predict Probability", "Actual Binary", "Stock Volatility",
                                              "Next Period Return"])

    # Save results to CSV
    df_results.to_csv("stock_predictions_results.csv", index=False)
    df_classification.to_csv("stock_predictions_classification.csv", index=False)

    print("Results saved successfully!")

elif state == 'train_RL':

    # Load sample stock data (Replace with actual data loading)
    stock_data = pd.read_csv("sample_stock_data.csv")  # Assume CSV file with test data

    # Train the RL agent
    trained_model = train_rl_agent(stock_data, max_periods=213, num_episodes=5000)
    # trained_model = train_rl_agent(stock_data, max_periods=213, num_episodes=5000)

    model_save_path = "trained_portfolio_rl_agent.zip"
    trained_model.save(model_save_path)
    print(f"Trained model saved to {model_save_path}")

    # Load the trained model (Optional)
    loaded_model = PPO.load(model_save_path)
    print("Loaded trained model successfully!")

    # Get averaged optimal parameters
    M = 100  # Number of samples to generate
    avg_short_ratio, avg_confidence_threshold = get_average_optimal_parameters(trained_model, stock_data,
                                                                               num_periods=10, M=M)

    print("Averaged Optimized Short Ratio:", avg_short_ratio)
    print("Averaged Optimized Confidence Threshold Low:", avg_confidence_threshold[0])
    print("Averaged Optimized Confidence Threshold High:", avg_confidence_threshold[1])

    # Save averaged optimal parameters to CSV
    optimal_params_df = pd.DataFrame({
        'Short Ratio': [avg_short_ratio],
        'Confidence Threshold Low': [avg_confidence_threshold[0]],
        'Confidence Threshold High': [avg_confidence_threshold[1]]
    })
    optimal_params_df.to_csv("optimal_parameters.csv", index=False)

    print("Optimal parameters saved to optimal_parameters.csv")

In [7]:
!pip install ace_tools



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
from itertools import product
portfolio_data = pd.read_csv("sample_stock_data.csv")  
# Extract relevant data from the uploaded dataset
stock_tickers = portfolio_data["Stock"].values
predicted_probs = portfolio_data["Predict Probability"].values
rolling_volatility = portfolio_data["Volatility"].values
sectors = portfolio_data["Sector"].values
actual_returns = portfolio_data["Next Period Return"].values  # To evaluate portfolio performance

# Define search space for confidence_threshold and short_ratio
confidence_thresholds = [(low, high) for low, high in product(np.arange(0.1, 0.8, 0.01), repeat=2) if low < high]
short_ratios = np.arange(0.0, 0.5, 0.01)

# Define function to evaluate Sharpe Ratio for a given parameter set
def evaluate_parameters(confidence_threshold, short_ratio):
    try:
        portfolio_value = 1000 
        portfolio_df = compute_portfolio_weights(stock_tickers, predicted_probs, rolling_volatility, sectors,
                                                 confidence_threshold=confidence_threshold, short_ratio=short_ratio,
                                                 annualize_volatility=False)
        # Compute portfolio return
        portfolio_return = (portfolio_df['Final Weight'] * portfolio_data['Next Period Return']).sum()
 
        # Update portfolio value
        portfolio_value *= (1 + portfolio_return)

        # Compute portfolio return and volatility
        # portfolio_return = np.sum(portfolio_df["Final Weight"].values * actual_returns)
        portfolio_volatility = np.std(portfolio_df['Final Weight'] * portfolio_data['Next Period Return'])
        sharpe_ratio = portfolio_return / (portfolio_volatility + 1e-6)  # Avoid division by zero
        # sharpe_ratio = portfolio_value
        # return sharpe_ratio, portfolio_return, portfolio_volatility
        return sharpe_ratio, portfolio_return
    except Exception:
        return -np.inf, 0  # Return worst possible Sharpe ratio if constraints fail

# Perform grid search
best_sharpe_ratio = -np.inf
best_params = None
results = []

for conf_thresh, short_r in product(confidence_thresholds, short_ratios):
    sharpe, ret = evaluate_parameters(conf_thresh, short_r)
    results.append((conf_thresh, short_r, sharpe, ret))
    if sharpe > best_sharpe_ratio:
        best_sharpe_ratio = sharpe
        best_params = (conf_thresh, short_r)

# Convert results to DataFrame for analysis
results_df = pd.DataFrame(results, columns=["Confidence Threshold", "Short Ratio", "Sharpe Ratio", "Return"])


# Return the best parameters found
best_params
# results_df

((np.float64(0.5799999999999997), np.float64(0.6199999999999998)),
 np.float64(0.0))

In [27]:

def compute_portfolio_weights_per_period(df, confidence_threshold=(0.4, 0.6), short_ratio=0.25, annualize_volatility=False):
    """
    Compute optimal portfolio weights for long-short portfolio ensuring total weight sums to 100%
    while maintaining the short ratio.

    Parameters:
    - stock_tickers (list): List of stock tickers.
    - predicted_probs (np.array): Predicted probability of stock moving up.
    - rolling_volatility (np.array): Rolling volatility (daily or annualized).
    - sectors (list): List of sector classifications for each stock.
    - confidence_threshold (tuple): Threshold (low, high) to filter out uncertain predictions.
    - short_ratio (float): Proportion of short exposure (between 0 and 50%).
    - annualize_volatility (bool): Whether to annualize daily volatility using sqrt(252).

    Returns:
    - pd.DataFrame: Portfolio allocation with normalized long-short weights.
    """
    portfolio_results = []

    for period, period_data in df.groupby("End Date"):
        stock_tickers = period_data["Stock"].values
        predicted_probs = period_data["Predict Probability"].values
        rolling_volatility = period_data["Volatility"].values
        sectors = period_data["Sector"].values

        # Convert probability into raw weights for long-short portfolio
        raw_weights = 2 * (predicted_probs - 0.5)

        # Apply confidence threshold: Only include confident stocks
        confidence_mask = (predicted_probs >= confidence_threshold[1]) | (predicted_probs <= confidence_threshold[0])
        filtered_weights = raw_weights * confidence_mask  # Zero out low-confidence predictions

        # Skip period if no stocks qualify
        if np.all(filtered_weights == 0):
            continue

        # Annualize volatility if required
        if annualize_volatility:
            rolling_volatility *= np.sqrt(252)

        # Apply Volatility Adjustment: Wi,Tadjusted = Wi,T / σi
        adjusted_weights = filtered_weights / rolling_volatility  # Scale by inverse volatility

        # Separate long and short portfolios based on short ratio
        long_weights = np.where(adjusted_weights > 0, adjusted_weights, 0)
        short_weights = np.where(adjusted_weights < 0, adjusted_weights, 0)

        # Normalize long and short weights so they sum to the target allocations
        long_sum = np.sum(long_weights)
        short_sum = np.abs(np.sum(short_weights))

        if long_sum > 0:
            long_weights *= (1 + short_ratio) / long_sum  # Scale long portfolio to (1 + short_ratio)
        if short_sum > 0:
            short_weights *= short_ratio / short_sum  # Scale short portfolio to -short_ratio

        # Compute final portfolio weights (long + short should sum to 100%)
        final_weights = long_weights + short_weights

        # Store results
        period_portfolio = pd.DataFrame({
            "End Date": period,
            "Stock": stock_tickers,
            "Sector": sectors,
            "Probability": predicted_probs,
            "Raw Weight": raw_weights,
            "Volatility": rolling_volatility,
            "Adjusted Weight": adjusted_weights,
            "Final Weight": final_weights
        })

        portfolio_results.append(period_portfolio)

    # Combine all periods into a single DataFrame
    portfolio_df = pd.concat(portfolio_results, ignore_index=True)
    return portfolio_df
portfolio_data = pd.read_csv("sample_stock_data.csv")  
portfolio_value = 1000 
confidence_threshold = (0.1, 0.66)
short_ratio = 0.49
portfolio_df = compute_portfolio_weights_per_period(portfolio_data, confidence_threshold=confidence_threshold, short_ratio=short_ratio,
                                     annualize_volatility=False)
print(f'portfolio_df={portfolio_df}')
# Compute portfolio return
portfolio_return = (portfolio_df['Final Weight'] * portfolio_data['Next Period Return']).sum()

print(f'portfolio_return = {portfolio_return}')
# Update portfolio value
portfolio_value *= (1 + portfolio_return)
print(f'portfolio_value = {portfolio_value}')
# Compute portfolio return and volatility
# portfolio_return = np.sum(portfolio_df["Final Weight"].values * actual_returns)
# portfolio_volatility = np.std(portfolio_df["Final Weight"].values * rolling_volatility)
# sharpe_ratio = portfolio_return / (portfolio_volatility + 1e-6)  # Avoid division by zero
sharpe_ratio = portfolio_value

portfolio_df=       End Date Stock                  Sector   Probability  Raw Weight  \
0     1/10/2008  AAPL              Technology  6.470000e-08   -1.000000   
1     1/10/2008  AMGN              Healthcare  5.655593e-01    0.131119   
2     1/10/2008  AMZN  Consumer Discretionary  1.365920e-03   -0.997268   
3     1/10/2008   AXP              Financials  1.000000e+00    1.000000   
4     1/10/2008    BA             Industrials  5.396600e-04   -0.998921   
...         ...   ...                     ...           ...         ...   
5959   9/8/2010    PG        Consumer Staples  4.061087e-02   -0.918778   
5960   9/8/2010   TRV              Financials  9.813436e-01    0.962687   
5961   9/8/2010   UNH              Healthcare  2.026568e-02   -0.959469   
5962   9/8/2010    VZ  Communication Services  9.999965e-01    0.999993   
5963   9/8/2010   WMT        Consumer Staples  2.013629e-02   -0.959727   

      Volatility  Adjusted Weight  Final Weight  
0       0.027976       -35.744920   

In [29]:
import sys
from SVERL_icml_2023.portfolio_DRL.data_function import *
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader
from SVERL_icml_2023.portfolio_DRL.create_model import *
from sklearn.model_selection import TimeSeriesSplit
from SVERL_icml_2023.shapley import Shapley
import shap
sys.path.append("E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023")
# from portfolio_DRL.create_model import StockPredictWrapper

# Example usage
input_dir = "E:\XAI\RL_with_SHAP\pythonProject1\SVERL_icml_2023\portfolio_DRL"  # Use the specified directory
ticker_file = os.path.join(input_dir, "tickers.txt")  # Text file containing tickers, one per line
factor_file = os.path.join(input_dir, "FF_factors.csv")  # CSV file containing factor returns
start_date = "2005-01-01"
end_date = "2024-12-31"
output_csv = os.path.join(input_dir, "stock_prices.csv")

try:
    tickers = load_tickers_from_file(ticker_file)
    price_data = download_stock_prices(tickers, start_date, end_date, output_csv)
    daily_returns = calculate_daily_returns(price_data, return_type='arithmetic')
    daily_returns.to_csv(os.path.join(input_dir, "daily_returns.csv"))
    factor_returns = read_factor_returns(factor_file, daily_returns)
    factor_returns.to_csv(os.path.join(input_dir, "aligned_factors.csv"))
    print("Factor returns alignment completed.")
except Exception as e:
    print(f"An error occurred: {e}")

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed

An error occurred: AAPL data is required to check the reference length.
